In [ ]:
import anndata as ad
import matplotlib.pyplot as plt
import metacells as mc
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp
import seaborn as sb

from math import hypot
from matplotlib.collections import LineCollection
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('svg')
sb.set_style("white")

In [ ]:
raw = ad.read_h5ad('output/scrna_db/oc_cells_with_md.h5ad')
# raw = mc.ut.slice(raw, vars=np.squeeze(np.asarray(np.sum(raw.X, axis = 0))) > 100)
# exe_cells = set(i for j in pd.read_table("../rabemb/exe_cells_fixed.txt", header=None).values for i in j)
raw = mc.ut.slice(raw, obs=raw.obs.t<9)

In [ ]:
mc.ut.set_name(raw, 'embs')
print(raw.shape)

In [ ]:
from tqdm import tqdm
# grep -E "RPS[0-9]+|RPL[0-9]+"  blasted_all_refseq_full | grep -o -E "X[A-Z]_[0-9]+.[0-9]" | uniq > ribosomal_refseq
with open("ribosomal_refseq", "r") as f:
    ribo = set(f.read().split('\n'))

# grep -E "MTMR*|MTRNR*|MTND*"  blasted_all_refseq_full | grep -o -E "X[A-Z]_[0-9]+.[0-9]" | uniq > mitochondrial_refseq
with open("mitochondrial_refseq", "r") as f:
    mito = set(f.read().split('\n'))

blast6 = pd.read_table('blasted_all_refseq6', header=None)
ribo_hs = set(blast6[blast6[1].isin(ribo)][0])
mito_hs = set(blast6[blast6[1].isin(mito)][0])

In [ ]:
import metacells.tools as tl

In [ ]:
ribo_hs |= set(mc.ut.slice(
    raw, 
    vars=tl.find_named_genes(
        raw,
        names=["RN18S"],
        patterns=["^RPS\d+", '^RPL\d+']
    )).var_names.to_list())

In [ ]:
mito_hs |= set(mc.ut.slice(
    raw, 
    vars=tl.find_named_genes(
        raw,
        patterns=["^MTMR*", '^MTRNR*', '^MTND*']
    )).var_names.to_list())

In [ ]:
cells_mito = [i[0] for i in (mc.ut.slice(raw, vars=raw.var_names.intersection(mito_hs)).X.sum(axis=1)/raw.X.sum(axis=1)).tolist()]
cells_ribo = [i[0] for i in (mc.ut.slice(raw, vars=raw.var_names.intersection(ribo_hs)).X.sum(axis=1)/raw.X.sum(axis=1)).tolist()]

In [ ]:
len([i for i in cells_mito if i > 0.05])

In [ ]:
len([i for i in cells_ribo if i > 0.4])

In [ ]:
cells_mito, cells_ribo = zip(*((i,j) for i,j in zip(cells_mito, cells_ribo) if i < 0.05 and j < 0.4))

In [ ]:
import seaborn as sns
sns.jointplot(x=cells_mito, y=cells_ribo, kind="hex")
plt.show()

In [ ]:
excluded_gene_names = ["NEATL", "ATPASE6", "XIST", "XIST_hg19", "MALATL", "TMSB4X", "T17H1","ATP6;ATP8;COX3", "GM20594_mm9",
                          "ND3", "COX2", "LOC114163077", "hotspot36379", "hotspot78915",
                          "T17H1","hotspot78914","hotspot95595","hotspot7136","hotspot7125",
                       'hotspot14535','hotspot39482','hotspot39483','hotspot45858','hotspot45859','hotspot106648','hotspot106649',
                      'RN18S',
                      ] + [i.upper() for i in [
    "Neat1", "Xist", "Malat1", "AK140265","AK018753","AK163440","DQ539915","AK131586","AK131579","AK142750","X57780","GU332589","BC071253"
]]
# Add mouse list

excluded_gene_patterns = [
#     '^IGJ', '^IGH', '^IGK', '^IGL', 'MT-*', "^MTMR*", '^MTRNR*', '^MTND*',  
    'ERCC*', 
#     'hotspot*', 
#     'LOC*'
]

In [ ]:
#add hotspots that match excluded_gene_patterns via blast
excluded_gene_names += ['hotspot525','hotspot526','hotspot2359','hotspot3137','hotspot4330','hotspot4331','hotspot4791','hotspot6988','hotspot6989','hotspot7352','hotspot7353','hotspot11913','hotspot13377','hotspot13378','hotspot13435','hotspot13992','hotspot16579','hotspot16580','hotspot17408','hotspot17409','hotspot17420','hotspot19005','hotspot19269','hotspot19498','hotspot19499','hotspot20171','hotspot22740','hotspot23925','hotspot23926','hotspot23932','hotspot27192','hotspot27193','hotspot28496','hotspot29480','hotspot29481','hotspot29544','hotspot29545','hotspot30067','hotspot30068','hotspot31816','hotspot31817','hotspot32422','hotspot32423','hotspot32596','hotspot32597','hotspot35020','hotspot35021','hotspot35261','hotspot35262','hotspot35273','hotspot35274','hotspot38557','hotspot38868','hotspot39330','hotspot39873','hotspot40161','hotspot41188','hotspot41189','hotspot41242','hotspot41243','hotspot42112','hotspot42113','hotspot42327','hotspot43721','hotspot43722','hotspot43784','hotspot44104','hotspot44105','hotspot44631','hotspot44632','hotspot44719','hotspot44720','hotspot46522','hotspot47882','hotspot47883','hotspot51944','hotspot51945','hotspot54962','hotspot55071','hotspot55915','hotspot55916','hotspot56444','hotspot56445','hotspot56446','hotspot58394','hotspot58395','hotspot61769','hotspot61770','hotspot62384','hotspot62385','hotspot63943','hotspot64097','hotspot64098','hotspot64955','hotspot64956','hotspot65034','hotspot65401','hotspot65402','hotspot67351','hotspot68679','hotspot68680','hotspot68927','hotspot68928','hotspot69148','hotspot69564','hotspot69617','hotspot70396','hotspot70397','hotspot71603','hotspot71631','hotspot71632','hotspot71686','hotspot71687','hotspot72434','hotspot72938','hotspot72939','hotspot72947','hotspot72948','hotspot73377','hotspot76789','hotspot76790','hotspot77954','hotspot78797','hotspot79312','hotspot79313','hotspot80943','hotspot80944','hotspot82719','hotspot82720','hotspot84164','hotspot84165','hotspot85262','hotspot85263','hotspot86033','hotspot86034','hotspot86231','hotspot89977','hotspot89978','hotspot90025','hotspot90048','hotspot90049','hotspot92121','hotspot92791','hotspot92792','hotspot92968','hotspot94959','hotspot95424','hotspot95455','hotspot95456','hotspot96713','hotspot96714','hotspot96743','hotspot97518','hotspot97519','hotspot97829','hotspot97830','hotspot97912','hotspot98084','hotspot98085','hotspot99245','hotspot99246','hotspot101106','hotspot101107','hotspot101729','hotspot102045','hotspot102362','hotspot102363','hotspot102806','hotspot102807','hotspot103293','hotspot104096','hotspot104097','hotspot104445','hotspot105449','hotspot105450','hotspot108764','hotspot109082','hotspot109083','hotspot109545','hotspot109546','hotspot111225','hotspot112355','hotspot113241','hotspot113717','hotspot113718','hotspot114328','hotspot114781','hotspot114820','hotspot114821','hotspot114843','hotspot114899','hotspot114900','hotspot115467','hotspot115597','hotspot115598','hotspot116568','hotspot117167',]

In [ ]:
%%time
mc.pl.analyze_clean_genes(raw,
                          excluded_gene_names=excluded_gene_names,
                          excluded_gene_patterns=excluded_gene_patterns,
                          random_seed=123456)

In [ ]:
%%time
mc.pl.pick_clean_genes(raw)

In [ ]:
%%time
raw.write('full85.h5ad')
full = raw

In [ ]:
%%time
properly_sampled_min_cell_total = 2600
properly_sampled_max_cell_total = 40000

total_umis_of_cells = mc.ut.get_o_numpy(full, name='__x__', sum=True)

plot = sb.distplot(total_umis_of_cells)
plot.set(xlabel='UMIs', ylabel='Density', yticks=[])
plot.axvline(x=properly_sampled_min_cell_total, color='darkgreen')
plot.axvline(x=properly_sampled_max_cell_total, color='crimson')

too_small_cells_count = sum(total_umis_of_cells < properly_sampled_min_cell_total)
too_large_cells_count = sum(total_umis_of_cells > properly_sampled_max_cell_total)

too_small_cells_percent = 100.0 * too_small_cells_count / len(total_umis_of_cells)
too_large_cells_percent = 100.0 * too_large_cells_count / len(total_umis_of_cells)

print(f"Will exclude %s (%.2f%%) cells with less than %s UMIs"
      % (too_small_cells_count,
         too_small_cells_percent,
         properly_sampled_min_cell_total))
print(f"Will exclude %s (%.2f%%) cells with more than %s UMIs"
      % (too_large_cells_count,
         too_large_cells_percent,
         properly_sampled_max_cell_total))

In [ ]:
np.median(total_umis_of_cells)

In [ ]:
%%time
properly_sampled_max_excluded_genes_fraction = 0.4

excluded_genes_data = mc.tl.filter_data(full, var_masks=['~clean_gene'])[0]
excluded_umis_of_cells = mc.ut.get_o_numpy(excluded_genes_data, name='__x__', sum=True)
excluded_fraction_of_umis_of_cells = excluded_umis_of_cells / total_umis_of_cells

plot = sb.distplot(excluded_fraction_of_umis_of_cells)
plot.set(xlabel='Fraction of excluded gene UMIs', ylabel='Density', yticks=[])
plot.axvline(x=properly_sampled_max_excluded_genes_fraction, color='crimson')

too_excluded_cells_count = sum(excluded_fraction_of_umis_of_cells > properly_sampled_max_excluded_genes_fraction)

too_excluded_cells_percent = 100.0 * too_excluded_cells_count / len(total_umis_of_cells)

print(f"Will exclude %s (%.2f%%) cells with more than %.2f%% excluded gene UMIs"
      % (too_excluded_cells_count,
         too_excluded_cells_percent,
         100.0 * properly_sampled_max_excluded_genes_fraction))

In [ ]:
%%time
mc.pl.analyze_clean_cells(
    full,
    properly_sampled_min_cell_total=properly_sampled_min_cell_total,
    properly_sampled_max_cell_total=properly_sampled_max_cell_total,
    properly_sampled_max_excluded_genes_fraction=properly_sampled_max_excluded_genes_fraction)

In [ ]:
%%time
mc.pl.pick_clean_cells(full)

In [ ]:
%%time
clean = mc.pl.extract_clean_data(full)

In [ ]:
min(clean.X.sum(axis=1))

In [ ]:
# suspect_gene_patterns = ['hotspot*']

# suspect_genes_mask = mc.tl.find_named_genes(clean, patterns=suspect_gene_patterns)

# good_hotspots = ['hotspot44767','hotspot8759','hotspot7747','hotspot75636',
#     'hotspot676','hotspot6935','hotspot6936','hotspot7586','hotspot7587','hotspot10202','hotspot13345','hotspot16218','hotspot16219','hotspot17420','hotspot29329','hotspot34336','hotspot38700','hotspot38701','hotspot50051','hotspot52464','hotspot52653','hotspot54734','hotspot54735','hotspot55771','hotspot57560','hotspot57561','hotspot59413','hotspot59414','hotspot66331','hotspot68243','hotspot68244','hotspot68310','hotspot68311','hotspot69564','hotspot70917','hotspot71631','hotspot71632','hotspot72016','hotspot72017','hotspot73144','hotspot74714','hotspot74715','hotspot82691','hotspot82692','hotspot87305','hotspot89837','hotspot90125','hotspot90126','hotspot93523','hotspot97072','hotspot97073','hotspot101106','hotspot101107','hotspot102362','hotspot102363','hotspot103293','hotspot105034','hotspot105212','hotspot105213','hotspot109665','hotspot112355','hotspot113580','hotspot113586','hotspot113587','hotspot113717','hotspot113718','hotspot114279','hotspot114797','hotspot116485']

# suspect_genes_mask[set(suspect_genes_mask.index) & set(good_hotspots)
# ] = False

# unknown_hotspots = sorted(clean.var_names[suspect_genes_mask])

In [ ]:
%%time
suspect_gene_names = ['PCNA', 'MKI67', 'TOP2A', 'HIST1H1D',
                      'FOS', 'HSP90AB1', 'HSPA1A','HSP5',
                      'ISG15', 'WARS' 'UBE2C', 'ND3',
    'IGHMBP2', 'IGLL1', 'IGLL5', 'IGLON5', 'NEAT1', 'TMSB4X', "NEATL",
    "ATPASE6", "XIST", "MALATL", "TMSB4X", "T17H1", "ATP6;ATP8;COX3", "GM20594_mm9", "ND3", "COX2",
    "NEAT1","TMSB4X",    
    
#     "RASGRF1", "MIR682_mm9", "GM15501_mm9", "LEVY_other_m", "DNAJA1P5_hg19", "SNR-2_other", "GM11978", "RPS27", "RPS27-2_otherCBX3P2_hg19",
    'HBB1', 'HBG1','HBG2', 'RN18S', 'HBZ',
    
#     'OLFR645',
    'ASPM', 'ATF7IP',
    'hotspot655',
                      
    # previously forbidden
    "ACI29_other_m", "AHSP", "ANKRD11", "ARHGAP27P1-BPTFP1-KPNA2P3_hg19", "ASIC1", "ATP5J", "ATP5MC3_hg19", "ATRX", "AURKA", "AURKB", "BCLAF1", "BOD1L1", "BRD4", "BUB1", "BUB1B", "C12H17ORF75_other_m", "C1H11orf31", "CCNB2", "CCND2", "CDC20;MPL", "CDCA2", "CDCA3", "CDCA7", "CDK1", "CENPC", "CENPE", "CENPF", "CEP128", "CHD4", "CHD7", "CKAP2L", "CKAP5", "CKS1_other_m", "CLEC2D_hg19", "CLSPN", "COX15_mm9", "COX16_mm9_m", "COX20_mm9", "COX4I2_other_m", "COX6A1", "COX6C_mm9", "COX7A2L_hg19", "COX7A2_mm9", "COX7C_mm9_m", "CROT", "DEPDC1B", "DGUOK", "DLGAP5", "DTL", "ECT2", "EGR1", "EIF3A", "EIF4G1", "EIF4G2", "ELOVL4", "F2RL1", "FADS1", "FAM110C", "FAM64A", "FAT1", "FLJ44635_hg19", "FOS", "GADD45A",  "GFPT2", "GM4925_mm9", "GM5176_mm9", "GM6607_mm9", "GNAI2", "HBB2", "HBG1", "HBG2", "HBZ", "HDAC2_mm9", "HMMR", "HS1BP3", "HS2ST1", "HS3ST2", "HS3ST3A1", "HS3ST3B1", "HS3ST3L_other", "HS3ST4", "HS3ST5", "HS3ST6", "HS6ST2", "HS6ST3", "HSBP1L1", "HSCB", "HSD11B1", "HSD11B1L", "HSD17B10", "HSD17B12", "HSD17B2", "HSD17B3", "HSD17B4", "HSD17B7", "HSD17B8", "HSD3B7", "HSDL1", "HSDL2", "HSF1", "HSF2", "HSF2BP", "HSF4", "HSF5", "HSH2D", "HSP20.1_other", "HSP90AA1", "HSP90AB1", "HSPA12A", "HSPA13", "HSPA14", "HSPA1A.S_other_m", "HSPA1B_mm9", "HSPA2", "HSPA4", "HSPA4L", "HSPA6", "HSPA8", "HSPA9", "HSPB11", "HSPB6", "HSPB7", "HSPB9", "HSPBAP1", "HSPBP1", "HSPD1.S_other", "HSPE1", "HSPH1", "HYLS1", "HYOU1", "IFI35", "IFI44", "IFI44L", "IFIH1", "IFIT2", "IFIT3", "IFIT5", "IFITM10_mm9", "IFITM1_mm9", "ISG15", "KIF11", "KIF14", "KIF18A", "KIF20A", "KIF20B", "KIF4A", "KIFC1", "KLRG2", "KMT2A", "KMT2C", "KNL1", "KNSTRN", "KTN1", "LOC100287467", "LOC100339314", "LOC100340645", "LOC100341037", "LOC100341713", "LOC100342635", "LOC100343039", "LOC100343268", "LOC100343557", "LOC100343802", "LOC100344007", "LOC100344156", "LOC100344388", "LOC100345882", "LOC100345978", "LOC100346597", "LOC100348336", "LOC100348796", "LOC100349084", "LOC100349527", "LOC100351037", "LOC100351519", "LOC100353361", "LOC100353654;LOC108176166", "LOC100353836", "LOC100356667", "LOC100357307", "LOC100357840", "LOC100357913", "LOC100358336", "LOC102723302", "LOC103345230", "LOC103346294", "LOC103347883", "LOC103349265", "LOC103349305", "LOC103969081", "LOC108176098", "LOC108176430", "LOC108176694", "LOC108176887", "LOC108177422", "MCM10", "MCM2", "MCM3", "MCM3AP", "MCM4", "MCM5", "MCM6", "MCM7", "MCM8", "MCM9", "MGA", "MIS18BP1", "MKI67", "MMD", "MPP7", "MSH6", "NASP", "NCAPD2", "NCAPG", "NCL", "NDC80", "NEFM", "NEK2", "NKTR", "NOP56", "NPM1", "NPM3", "NUCKS1", "NUSAP1", "PARP1", "PCLO", "PCNA", "PDIA6", "PIF1", "PLK1", "POR", "PPIA", "PPM1J", "PPP1R12B", "PRDX1", "PRR11", "PRRC2C", "PSRC1", "RBBP6", "RN18S", "RPL10", "RPL17_mm9_m", "RPL18A", "RPL19P12_hg19", "RPL31_other_m", "RPL41_mm9", "RPL8", "RPS11_other", "RPS12", "RPS6-PS4_mm9", "RRM2", "RS5-8S1_other", "RSL1D1", "SESN2", "SGO2", "SMARCC1", "SMC1A", "SMC1B", "SMC2", "SMC3", "SMC4", "SMC4.L_other", "SMC5", "SMC6", "SPAG5", "SRRM2", "SRSF2", "THRAP3", "TNFRSF21", "TNRC6B", "TOP1", "TOP2A", "TPX2", "TSC1", "TTK", "UAP1", "UBE2C", "UHMK1", "UNC80", "ZC3H13", "hotspot100013", "hotspot100056", "hotspot100078", "hotspot100079", "hotspot100118", "hotspot100369", "hotspot100383", "hotspot100432", "hotspot1005", "hotspot100646", "hotspot100884", "hotspot100909", "hotspot101082", "hotspot101087", "hotspot101241", "hotspot101363", "hotspot101596", "hotspot101711", "hotspot102506", "hotspot10265", "hotspot102708", "hotspot10272", "hotspot102738", "hotspot103428", "hotspot103536", "hotspot103812", "hotspot103816", "hotspot104274", "hotspot10439", "hotspot10455", "hotspot104580", "hotspot104604", "hotspot104902", "hotspot105014", "hotspot10512", "hotspot105207", "hotspot105296", "hotspot105343", "hotspot10541", "hotspot105482", "hotspot10561", "hotspot105657", "hotspot105694", "hotspot105808", "hotspot105821", "hotspot106060", "hotspot106084", "hotspot106836", "hotspot106931", "hotspot107093", "hotspot107284", "hotspot107513", "hotspot107741", "hotspot107812", "hotspot108405", "hotspot108908", "hotspot109392", "hotspot109585", "hotspot110083", "hotspot110117", "hotspot11038", "hotspot110426", "hotspot110561", "hotspot110741", "hotspot111178", "hotspot111371", "hotspot11160", "hotspot111635", "hotspot111652", "hotspot111980", "hotspot112237", "hotspot112370", "hotspot112372", "hotspot112784", "hotspot112809", "hotspot113063", "hotspot113114", "hotspot113158", "hotspot113176", "hotspot113225", "hotspot1134", "hotspot113441", "hotspot113442", "hotspot113714", "hotspot113815", "hotspot113949", "hotspot114024", "hotspot114053", "hotspot114661", "hotspot114833", "hotspot115076", "hotspot115156", "hotspot115832", "hotspot115844", "hotspot115931", "hotspot115978", "hotspot11652", "hotspot116563", "hotspot116665", "hotspot116740", "hotspot116752", "hotspot116786", "hotspot117009", "hotspot117162", "hotspot117201", "hotspot117214", "hotspot117275", "hotspot12088", "hotspot121", "hotspot12192", "hotspot12264", "hotspot12353", "hotspot12494", "hotspot12697", "hotspot12820", "hotspot12854", "hotspot12956", "hotspot12972", "hotspot13030", "hotspot13075", "hotspot13128", "hotspot13401", "hotspot13422", "hotspot13742", "hotspot13840", "hotspot1402", "hotspot1406", "hotspot14248", "hotspot14271", "hotspot14371", "hotspot14417", "hotspot14584", "hotspot1498", "hotspot15389", "hotspot15520", "hotspot15638", "hotspot15986", "hotspot15995", "hotspot16388", "hotspot16745", "hotspot16856", "hotspot17351", "hotspot17595", "hotspot17610", "hotspot17750", "hotspot17771", "hotspot18204", "hotspot18272", "hotspot19546", "hotspot19916", "hotspot20137", "hotspot20272", "hotspot20386", "hotspot20391", "hotspot204", "hotspot20471", "hotspot20555", "hotspot20685", "hotspot2078", "hotspot21194", "hotspot21579", "hotspot22514", "hotspot22579", "hotspot22689", "hotspot22955", "hotspot23037", "hotspot23109", "hotspot2317", "hotspot23255", "hotspot23379", "hotspot23405", "hotspot23792", "hotspot24372", "hotspot24493", "hotspot24528", "hotspot24706", "hotspot24937", "hotspot25060", "hotspot252", "hotspot25268", "hotspot25304", "hotspot25313", "hotspot25381", "hotspot25511", "hotspot25553", "hotspot25646", "hotspot2604", "hotspot26133", "hotspot26524", "hotspot27014", "hotspot27027", "hotspot27334", "hotspot27367", "hotspot27708", "hotspot2781", "hotspot27891", "hotspot28144", "hotspot2822", "hotspot28355", "hotspot28551", "hotspot28594", "hotspot28787", "hotspot28886", "hotspot28910", "hotspot29471", "hotspot29492", "hotspot29716", "hotspot30205", "hotspot30301", "hotspot30572", "hotspot3071", "hotspot31092", "hotspot31160", "hotspot31469", "hotspot31530", "hotspot31782", "hotspot32260", "hotspot326", "hotspot32838", "hotspot3291", "hotspot32983", "hotspot33009", "hotspot33107", "hotspot33678", "hotspot33713", "hotspot33893", "hotspot34047", "hotspot34425", "hotspot34450", "hotspot34542", "hotspot34564", "hotspot34870", "hotspot34943", "hotspot3557", "hotspot36274", "hotspot36381", "hotspot36477", "hotspot365", "hotspot36706", "hotspot36841", "hotspot37", "hotspot37004", "hotspot37262", "hotspot37411", "hotspot37630", "hotspot37642", "hotspot37857", "hotspot38090", "hotspot38410", "hotspot38497", "hotspot38569", "hotspot3916", "hotspot39196", "hotspot39495", "hotspot39920", "hotspot40150", "hotspot40282", "hotspot40361", "hotspot40395", "hotspot40504", "hotspot41415", "hotspot4152", "hotspot41799", "hotspot41804", "hotspot41970", "hotspot42499", "hotspot42899", "hotspot42964", "hotspot4303", "hotspot43243", "hotspot43326", "hotspot43869", "hotspot4391", "hotspot44139", "hotspot44162", "hotspot44203", "hotspot44242", "hotspot4429", "hotspot44564", "hotspot44575", "hotspot44613", "hotspot44626", "hotspot44646", "hotspot44839", "hotspot44941", "hotspot45364", "hotspot46027", "hotspot46071", "hotspot46345", "hotspot46623", "hotspot46790", "hotspot47096", "hotspot47713", "hotspot47716", "hotspot47737", "hotspot47880", "hotspot47926", "hotspot48270", "hotspot48417", "hotspot48616", "hotspot48734", "hotspot49052", "hotspot49134", "hotspot49418", "hotspot49548", "hotspot49865", "hotspot49961", "hotspot5006", "hotspot50060", "hotspot50104", "hotspot50263", "hotspot50421", "hotspot50432", "hotspot50731", "hotspot50752", "hotspot51602", "hotspot51745", "hotspot51909", "hotspot52026", "hotspot52332", "hotspot52762", "hotspot53983", "hotspot54206", "hotspot54357", "hotspot54364", "hotspot5442", "hotspot54485", "hotspot54565", "hotspot54765", "hotspot54913", "hotspot55055", "hotspot55084", "hotspot55249", "hotspot55344", "hotspot55687", "hotspot560", "hotspot5601", "hotspot56070", "hotspot5638", "hotspot57045", "hotspot57064", "hotspot57125", "hotspot57142", "hotspot57164", "hotspot57603", "hotspot57753", "hotspot57756", "hotspot57956", "hotspot58276", "hotspot58285", "hotspot58583", "hotspot58618", "hotspot58945", "hotspot59024", "hotspot59091", "hotspot59149", "hotspot5927", "hotspot59615", "hotspot59833", "hotspot60317", "hotspot60329", "hotspot6037", "hotspot60521", "hotspot60733", "hotspot61092", "hotspot61814", "hotspot61852", "hotspot62052", "hotspot62143", "hotspot62181", "hotspot62802", "hotspot6291", "hotspot63057", "hotspot63104", "hotspot63234", "hotspot63893", "hotspot63959", "hotspot64118", "hotspot64434", "hotspot64472", "hotspot64545", "hotspot649", "hotspot64979", "hotspot65067", "hotspot65083", "hotspot65314", "hotspot65410", "hotspot65638", "hotspot65664", "hotspot65676", "hotspot65839", "hotspot66011", "hotspot66193", "hotspot66589", "hotspot66745", "hotspot66857", "hotspot67007", "hotspot67428", "hotspot67927", "hotspot68571", "hotspot69102", "hotspot69189", "hotspot69466", "hotspot69550", "hotspot69814", "hotspot70022", "hotspot70264", "hotspot70628", "hotspot70791", "hotspot70848", "hotspot71218", "hotspot71689", "hotspot71760", "hotspot71817", "hotspot71828", "hotspot71922", "hotspot71953", "hotspot72037", "hotspot72046", "hotspot72162", "hotspot72384", "hotspot72809", "hotspot73241", "hotspot73242", "hotspot73331", "hotspot7336", "hotspot73659", "hotspot73707", "hotspot74202", "hotspot74212", "hotspot74468", "hotspot74552", "hotspot74678", "hotspot74823", "hotspot75070", "hotspot752", "hotspot75399", "hotspot75459", "hotspot75664", "hotspot75667", "hotspot75884", "hotspot76179", "hotspot76229", "hotspot76256", "hotspot76373", "hotspot76529", "hotspot76711", "hotspot77424", "hotspot77435", "hotspot77654", "hotspot77775", "hotspot77797", "hotspot7786", "hotspot78291", "hotspot78437", "hotspot78559", "hotspot78708", "hotspot78732", "hotspot78920", "hotspot79095", "hotspot79167", "hotspot79182", "hotspot79627", "hotspot79815", "hotspot7983", "hotspot79942", "hotspot80020", "hotspot80130", "hotspot80431", "hotspot80566", "hotspot80810", "hotspot80811", "hotspot81289", "hotspot81790", "hotspot81915", "hotspot82419", "hotspot82471", "hotspot82559", "hotspot82873", "hotspot82926", "hotspot83448", "hotspot8356", "hotspot83618", "hotspot83805", "hotspot84029", "hotspot8411", "hotspot8429", "hotspot84424", "hotspot84430", "hotspot84921", "hotspot84965", "hotspot85474", "hotspot85537", "hotspot85855", "hotspot86623", "hotspot8666", "hotspot87133", "hotspot87257", "hotspot8759", "hotspot88267", "hotspot88479", "hotspot89287", "hotspot89291", "hotspot89681", "hotspot89886", "hotspot89956", "hotspot90288", "hotspot90325", "hotspot90384", "hotspot90570", "hotspot90663", "hotspot90839", "hotspot91297", "hotspot91443", "hotspot91663", "hotspot91770", "hotspot91878", "hotspot9197", "hotspot92283", "hotspot92610", "hotspot93056", "hotspot93257", "hotspot93388", "hotspot93468", "hotspot93483", "hotspot9409", "hotspot94297", "hotspot94482", "hotspot94500", "hotspot94553", "hotspot94554", "hotspot94576", "hotspot9521", "hotspot95470", "hotspot95474", "hotspot95551", "hotspot95609", "hotspot95651", "hotspot95850", "hotspot9586", "hotspot96093", "hotspot96133", "hotspot96716", "hotspot9708", "hotspot97296", "hotspot9756", "hotspot97913", "hotspot98260", "hotspot98280", "hotspot98401", "hotspot98447", "hotspot98601", "hotspot98760", "hotspot98912", "hotspot98995", "hotspot99215", "hotspot99372", "hotspot99825"
#     "HES1",
    # correlated with previously forbidden
    "hotspot37294",
    "hotspot3322", "hotspot42010", "hotspot21022","hotspot37294"
                      
    'ATP5J','C1H11orf31','CDC20;MPL','HMMR','LOC100348796','LOC103347883','LOC103349305','LOC108176694',
    'LOC108176887','MCM3','MCM4','NEFM','AURKB','BUB1','BUB1B','CCNB2','CDCA2','CDCA3','CENPC',
    'CKAP2L', 'CKAP5','DEPDC1B','DLGAP5','ECT2','FAM64A','HYLS1','KIF11','KIF14','KIF18A','KIF20A',
    'KIF20B','KIF4A','KIFC1','KNSTRN','MIS18BP1','NCAPD2','NCAPG','NDC80','NEK2','PIF1',
    'PRR11','PSRC1','SGO2','SPAG5','TTK', 'CDK1',
    'CENPE','CENPF','MKI67','NUSAP1','PLK1','TOP2A','TPX2','UBE2C',
    'LOC100343557','LOC100343802','EIF5A','LOC100353361',
    'LOC100340645','LOC100348336','LOC100357840','LOC108177422',
    'ATP6;ATP8;COX3','COX2','LOC114163077','ND3','T17H1','BRD4','EIF3A','EIF4G2',
    'HSP90AA1','LOC100344156','LOC100344388','LOC100345978','LOC100353836',
    'LOC100357913','LOC108176098','NCL','PRRC2C','TOP1','CDCA7',
    'DTL','PCNA','CCND2','FADS1','MMD',
    'NPM3','RRM2',
        'EFCAB',
                     
    #prev 2203 exclusion:
    'ACI29_other_m', 'ACRA1_other_m', 'ACTE1_mm9_m', 'AHSP', 'ANKRD11', 'ARHGAP27P1-BPTFP1-KPNA2P3_hg19', 'ASIC1', 'ATP5J', 'ATP5MC3_hg19', 'ATRX', 'AURKA', 'AURKB', 'BCLAF1', 'BEX3', 'BOD1L1', 'BRD4', 'BUB1', 'BUB1B', 'C12H17ORF75_other_m', 'C1H11orf31', 'CCNB2', 'CCND2', 'CDC20;MPL', 'CDCA2', 'CDCA3', 'CDCA7', 'CDK1', 'CENPC', 'CENPE', 'CENPF', 'CENPV', 'CEP128', 'CHD4', 'CHD7', 'CKAP2L', 'CKAP5', 'CKS1_other_m', 'CLEC2D_hg19', 'CLSPN', 'COL6A5', 'COQ2', 'COQ3;PNISR', 'COX15_mm9', 'COX16_mm9_m', 'COX20_mm9', 'COX4I2_other_m', 'COX6A1', 'COX6C_mm9', 'COX7A2L_hg19', 'COX7A2_mm9', 'COX7B_mm9_m', 'COX7C_mm9_m', 'CPN10_other', 'CROT', 'DEPDC1B', 'DGUOK', 'DLGAP5', 'DTL', 'DUSP13', 'ECT2', 'EEF1A1', 'EEF1B2', 'EFCAB5', 'EGR1', 'EIF3A', 'EIF4G1', 'EIF4G2', 'EIF5A', 'ELOVL4', 'F2RL1', 'FADS1', 'FAM110C', 'FAM60A', 'FAM64A', 'FAT1', 'FLJ44635_hg19', 'FOS', 'GADD45A', 'GFPT2', 'GM15501_mm9', 'GM4925_mm9', 'GM5069_mm9_m', 'GM5176_mm9', 'GM6607_mm9', 'GNAI2', 'GSTP1', 'HBB2', 'HBG1', 'HBG2', 'HBZ', 'HDAC2_mm9', 'HMMR', 'HS1BP3', 'HS2ST1', 'HS3ST2', 'HS3ST3A1', 'HS3ST3B1', 'HS3ST3L_other', 'HS3ST4', 'HS3ST5', 'HS3ST6', 'HS6ST2', 'HS6ST3', 'HSBP1L1', 'HSCB', 'HSD11B1', 'HSD11B1L', 'HSD17B10', 'HSD17B12', 'HSD17B2', 'HSD17B3', 'HSD17B4', 'HSD17B7', 'HSD17B8', 'HSD3B7', 'HSDL1', 'HSDL2', 'HSF1', 'HSF2', 'HSF2BP', 'HSF4', 'HSF5', 'HSH2D', 'HSP20.1_other', 'HSP90AA1', 'HSP90AB1', 'HSPA12A', 'HSPA13', 'HSPA14', 'HSPA1A.S_other_m', 'HSPA1B_mm9', 'HSPA2', 'HSPA4', 'HSPA4L', 'HSPA6', 'HSPA8', 'HSPB11', 'HSPB6', 'HSPB7', 'HSPB9', 'HSPBAP1', 'HSPBP1', 'HSPD1.S_other', 'HSPE1', 'HSPH1', 'HYLS1', 'HYOU1', 'IFI35', 'IFI44', 'IFI44L', 'IFIH1', 'IFIT2', 'IFIT3', 'IFIT5', 'IFITM10_mm9', 'IFITM1_mm9', 'ISG15', 'KDM5B', 'KHDRBS2', 'KIF11', 'KIF14', 'KIF18A', 'KIF20A', 'KIF20B', 'KIF4A', 'KIFC1', 'KLRG2', 'KMT2A', 'KMT2C', 'KNL1', 'KNSTRN', 'KTN1', 'LARP7', 'LDHC_mm9_m', 'LIN28_other_m', 'LOC100287467', 'LOC100313593', 'LOC100338112', 'LOC100339133', 'LOC100339314', 'LOC100340645', 'LOC100341037', 'LOC100341713', 'LOC100342501', 'LOC100342635', 'LOC100343039', 'LOC100343268', 'LOC100343557', 'LOC100343802', 'LOC100343982', 'LOC100344007', 'LOC100344156', 'LOC100344388', 'LOC100345882', 'LOC100345978', 'LOC100346597', 'LOC100348336', 'LOC100348796', 'LOC100349084', 'LOC100349527', 'LOC100351037', 'LOC100351519', 'LOC100353361', 'LOC100353654;LOC108176166', 'LOC100353836', 'LOC100356667', 'LOC100357307', 'LOC100357840', 'LOC100357913', 'LOC100358241', 'LOC100358336', 'LOC102723302', 'LOC103345230', 'LOC103346294', 'LOC103346930', 'LOC103347883', 'LOC103349265', 'LOC103349305', 'LOC103969081', 'LOC106105278;LOC106116970', 'LOC108176098', 'LOC108176430', 'LOC108176694', 'LOC108176887', 'LOC108177422', 'LOC108178646', 'MARCKSL1', 'MCM10', 'MCM2', 'MCM3', 'MCM3AP', 'MCM4', 'MCM5', 'MCM6', 'MCM7', 'MCM8', 'MCM9', 'MGA', 'MIR682_mm9', 'MIS18BP1', 'MKI67', 'MMD', 'MPP7', 'MSH6', 'MYO1F', 'NASP', 'NCAPD2', 'NCAPG', 'NCL', 'NDC80', 'NEFM', 'NEK2', 'NKTR', 'NOP56', 'NOS3', 'NPM1', 'NPM3', 'NSD1', 'NUCKS1', 'NUSAP1', 'OLFR645', 'PARP1', 'PCLO', 'PCNA', 'PDIA6', 'PFN1', 'PIF1', 'PLK1', 'POR', 'PPIA', 'PPIAL4A_hg19_m', 'PPM1J', 'PPP1R12B', 'PRDX1', 'PRDX6', 'PRR11', 'PRRC2C', 'PSIP1', 'PSRC1', 'RBBP6', 'RIF1', 'RN18S', 'RPL10', 'RPL11', 'RPL17_mm9_m', 'RPL18A', 'RPL19', 'RPL19P12_hg19', 'RPL21', 'RPL23B_other', 'RPL27A', 'RPL31', 'RPL31_other_m', 'RPL41_mm9', 'RPL7', 'RPL8', 'RPLP1', 'RPS-24_other', 'RPS11_other', 'RPS12', 'RPS20', 'RPS6-PS4_mm9', 'RRM2', 'RS5-8S1_other', 'RSF1', 'RSL1D1', 'SESN2', 'SGO2', 'SLC25A6_hg19', 'SMARCC1', 'SMC1A', 'SMC1B', 'SMC2', 'SMC3', 'SMC4', 'SMC4.L_other', 'SMC5', 'SMC6', 'SPAG5', 'SREK1', 'SRRM2', 'SRSF2', 'STIM1', 'THRAP3', 'TNFRSF21', 'TNRC6B', 'TOP1', 'TOP2A', 'TP53', 'TPT1', 'TPX2', 'TSC1', 'TTK', 'UAP1', 'UBE2C', 'UHMK1', 'UNC80', 'YBX3', 'ZC3H13', 'hotspot100013', 'hotspot100056', 'hotspot100078', 'hotspot100079', 'hotspot100118', 'hotspot100369', 'hotspot100383', 'hotspot100432', 'hotspot10048', 'hotspot1005', 'hotspot100610', 'hotspot100646', 'hotspot100684', 'hotspot100884', 'hotspot100909', 'hotspot100941', 'hotspot101000', 'hotspot101082', 'hotspot101087', 'hotspot101221', 'hotspot101241', 'hotspot101363', 'hotspot101596', 'hotspot101711', 'hotspot101986', 'hotspot102344', 'hotspot102506', 'hotspot10265', 'hotspot102693', 'hotspot102708', 'hotspot102714', 'hotspot10272', 'hotspot102738', 'hotspot10285', 'hotspot102996', 'hotspot103428', 'hotspot103459', 'hotspot103536', 'hotspot103597', 'hotspot103627', 'hotspot103659', 'hotspot103724', 'hotspot103725', 'hotspot103759', 'hotspot103784', 'hotspot103812', 'hotspot103816', 'hotspot103885', 'hotspot104110', 'hotspot10418', 'hotspot104184', 'hotspot104274', 'hotspot104300', 'hotspot104317', 'hotspot10439', 'hotspot104393', 'hotspot10455', 'hotspot104580', 'hotspot104582', 'hotspot104604', 'hotspot104639', 'hotspot1048', 'hotspot104902', 'hotspot104918', 'hotspot105014', 'hotspot105093', 'hotspot10512', 'hotspot105180', 'hotspot105207', 'hotspot105248', 'hotspot105296', 'hotspot105343', 'hotspot10541', 'hotspot105482', 'hotspot10561', 'hotspot105657', 'hotspot105694', 'hotspot105706', 'hotspot105808', 'hotspot105821', 'hotspot106057', 'hotspot106060', 'hotspot106082', 'hotspot106084', 'hotspot106292', 'hotspot106397', 'hotspot106632', 'hotspot106664', 'hotspot10681', 'hotspot106836', 'hotspot10684', 'hotspot10685', 'hotspot10686', 'hotspot106931', 'hotspot106947', 'hotspot10695', 'hotspot107093', 'hotspot107284', 'hotspot107341', 'hotspot107508', 'hotspot107513', 'hotspot10758', 'hotspot10764', 'hotspot107741', 'hotspot107812', 'hotspot107906', 'hotspot108405', 'hotspot108624', 'hotspot10864', 'hotspot108797', 'hotspot108807', 'hotspot108908', 'hotspot108909', 'hotspot109043', 'hotspot109109', 'hotspot109285', 'hotspot109336', 'hotspot109392', 'hotspot109580', 'hotspot109585', 'hotspot109644', 'hotspot109714', 'hotspot109817', 'hotspot109946', 'hotspot109968', 'hotspot110031', 'hotspot110083', 'hotspot110117', 'hotspot110250', 'hotspot11038', 'hotspot110426', 'hotspot110561', 'hotspot110580', 'hotspot110582', 'hotspot110583', 'hotspot110707', 'hotspot110741', 'hotspot111178', 'hotspot111188', 'hotspot11122', 'hotspot111371', 'hotspot111411', 'hotspot111532', 'hotspot11160', 'hotspot111635', 'hotspot111652', 'hotspot111716', 'hotspot111727', 'hotspot111735', 'hotspot111955', 'hotspot111980', 'hotspot112237', 'hotspot112370', 'hotspot112372', 'hotspot11239', 'hotspot112632', 'hotspot112739', 'hotspot112760', 'hotspot112784', 'hotspot112806', 'hotspot112809', 'hotspot11283', 'hotspot113009', 'hotspot113063', 'hotspot113114', 'hotspot113158', 'hotspot113176', 'hotspot113188', 'hotspot113203', 'hotspot113225', 'hotspot113330', 'hotspot1134', 'hotspot113441', 'hotspot113442', 'hotspot113582', 'hotspot113651', 'hotspot113683', 'hotspot113714', 'hotspot113815', 'hotspot113899', 'hotspot113949', 'hotspot113972', 'hotspot114024', 'hotspot114053', 'hotspot114548', 'hotspot114661', 'hotspot11483', 'hotspot114833', 'hotspot114972', 'hotspot115076', 'hotspot115156', 'hotspot115222', 'hotspot115312', 'hotspot115428', 'hotspot115493', 'hotspot115773', 'hotspot115832', 'hotspot115844', 'hotspot115931', 'hotspot115978', 'hotspot116210', 'hotspot116271', 'hotspot116291', 'hotspot116493', 'hotspot11652', 'hotspot116551', 'hotspot116563', 'hotspot116665', 'hotspot116670', 'hotspot116699', 'hotspot116713', 'hotspot116740', 'hotspot116752', 'hotspot116786', 'hotspot117009', 'hotspot117162', 'hotspot117165', 'hotspot117201', 'hotspot117214', 'hotspot117275', 'hotspot11793', 'hotspot12024', 'hotspot12088', 'hotspot121', 'hotspot12171', 'hotspot12192', 'hotspot12264', 'hotspot12275', 'hotspot12310', 'hotspot12353', 'hotspot12480', 'hotspot12494', 'hotspot12697', 'hotspot12724', 'hotspot12820', 'hotspot12854', 'hotspot12930', 'hotspot12956', 'hotspot12972', 'hotspot1298', 'hotspot13030', 'hotspot13075', 'hotspot13128', 'hotspot13178', 'hotspot13398', 'hotspot13401', 'hotspot13422', 'hotspot13742', 'hotspot13840', 'hotspot1402', 'hotspot14030', 'hotspot14044', 'hotspot1406', 'hotspot14197', 'hotspot14248', 'hotspot14271', 'hotspot14371', 'hotspot14417', 'hotspot14584', 'hotspot14624', 'hotspot14740', 'hotspot14839', 'hotspot1498', 'hotspot15389', 'hotspot15520', 'hotspot15556', 'hotspot15638', 'hotspot15769', 'hotspot15986', 'hotspot15995', 'hotspot16388', 'hotspot16560', 'hotspot16745', 'hotspot1677', 'hotspot16856', 'hotspot17094', 'hotspot17260', 'hotspot17320', 'hotspot17351', 'hotspot17402', 'hotspot17439', 'hotspot17528', 'hotspot17595', 'hotspot17610', 'hotspot17750', 'hotspot17771', 'hotspot17824', 'hotspot18115', 'hotspot18204', 'hotspot18272', 'hotspot18288', 'hotspot18382', 'hotspot18411', 'hotspot18461', 'hotspot1848', 'hotspot18627', 'hotspot19095', 'hotspot19186', 'hotspot19289', 'hotspot19361', 'hotspot1940', 'hotspot19513', 'hotspot19546', 'hotspot1960', 'hotspot19662', 'hotspot19811', 'hotspot19834', 'hotspot19916', 'hotspot19960', 'hotspot19982', 'hotspot19988', 'hotspot20137', 'hotspot20272', 'hotspot20386', 'hotspot20391', 'hotspot204', 'hotspot20471', 'hotspot20555', 'hotspot20647', 'hotspot20685', 'hotspot20753', 'hotspot2078', 'hotspot21022', 'hotspot21034', 'hotspot21194', 'hotspot21559', 'hotspot21579', 'hotspot21622', 'hotspot21736', 'hotspot21889', 'hotspot21897', 'hotspot2226', 'hotspot22514', 'hotspot22579', 'hotspot22625', 'hotspot22689', 'hotspot22797', 'hotspot22955', 'hotspot23037', 'hotspot2305', 'hotspot23109', 'hotspot23121', 'hotspot2317', 'hotspot23204', 'hotspot23255', 'hotspot23273', 'hotspot23297', 'hotspot233', 'hotspot23379', 'hotspot23405', 'hotspot23480', 'hotspot23585', 'hotspot23792', 'hotspot23888', 'hotspot24033', 'hotspot24224', 'hotspot24316', 'hotspot24372', 'hotspot24438', 'hotspot24493', 'hotspot24528', 'hotspot24706', 'hotspot24767', 'hotspot24798', 'hotspot2489', 'hotspot24937', 'hotspot24962', 'hotspot25060', 'hotspot25066', 'hotspot25151', 'hotspot252', 'hotspot25247', 'hotspot25268', 'hotspot25304', 'hotspot25313', 'hotspot25381', 'hotspot25511', 'hotspot25553', 'hotspot25646', 'hotspot2586', 'hotspot25897', 'hotspot2604', 'hotspot26133', 'hotspot26442', 'hotspot26524', 'hotspot26776', 'hotspot26810', 'hotspot26855', 'hotspot27014', 'hotspot27027', 'hotspot27334', 'hotspot27367', 'hotspot27448', 'hotspot27594', 'hotspot27708', 'hotspot2781', 'hotspot27891', 'hotspot28057', 'hotspot28144', 'hotspot28163', 'hotspot2822', 'hotspot28355', 'hotspot28520', 'hotspot28551', 'hotspot28594', 'hotspot28787', 'hotspot28816', 'hotspot28886', 'hotspot28910', 'hotspot29471', 'hotspot29492', 'hotspot29716', 'hotspot29753', 'hotspot29839', 'hotspot29846', 'hotspot30205', 'hotspot30301', 'hotspot30308', 'hotspot30320', 'hotspot30342', 'hotspot30572', 'hotspot3071', 'hotspot30908', 'hotspot31092', 'hotspot31160', 'hotspot31213', 'hotspot31339', 'hotspot31469', 'hotspot31473', 'hotspot31530', 'hotspot31782', 'hotspot31835', 'hotspot31930', 'hotspot31980', 'hotspot32083', 'hotspot32092', 'hotspot32176', 'hotspot32260', 'hotspot32530', 'hotspot326', 'hotspot32610', 'hotspot3274', 'hotspot32801', 'hotspot32837', 'hotspot32838', 'hotspot32864', 'hotspot3291', 'hotspot32983', 'hotspot33009', 'hotspot33069', 'hotspot33107', 'hotspot33188', 'hotspot3322', 'hotspot33678', 'hotspot33713', 'hotspot33848', 'hotspot33893', 'hotspot33895', 'hotspot34047', 'hotspot3406', 'hotspot34177', 'hotspot34230', 'hotspot34347', 'hotspot34425', 'hotspot34450', 'hotspot34465', 'hotspot34530', 'hotspot34542', 'hotspot34564', 'hotspot34714', 'hotspot34870', 'hotspot34943', 'hotspot35548', 'hotspot3557', 'hotspot35793', 'hotspot36274', 'hotspot36381', 'hotspot36477', 'hotspot365', 'hotspot36706', 'hotspot36709', 'hotspot36739', 'hotspot3681', 'hotspot36841', 'hotspot36908', 'hotspot37', 'hotspot37004', 'hotspot37020', 'hotspot37262', 'hotspot37382', 'hotspot37411', 'hotspot37630', 'hotspot37642', 'hotspot37664', 'hotspot37807', 'hotspot37857', 'hotspot37958', 'hotspot38089', 'hotspot38090', 'hotspot38333', 'hotspot38410', 'hotspot38497', 'hotspot38569', 'hotspot39107', 'hotspot3916', 'hotspot39196', 'hotspot39476', 'hotspot39495', 'hotspot3975', 'hotspot39882', 'hotspot39920', 'hotspot40116', 'hotspot40150', 'hotspot40282', 'hotspot40288', 'hotspot40361', 'hotspot40395', 'hotspot40504', 'hotspot40625', 'hotspot40913', 'hotspot41142', 'hotspot41339', 'hotspot41415', 'hotspot4152', 'hotspot41628', 'hotspot41799', 'hotspot41804', 'hotspot41970', 'hotspot42010', 'hotspot42022', 'hotspot42081', 'hotspot42121', 'hotspot42123', 'hotspot42139', 'hotspot42146', 'hotspot42499', 'hotspot42512', 'hotspot42899', 'hotspot42964', 'hotspot4298', 'hotspot4303', 'hotspot4321', 'hotspot43243', 'hotspot43326', 'hotspot43695', 'hotspot43761', 'hotspot43869', 'hotspot4391', 'hotspot43912', 'hotspot44069', 'hotspot44139', 'hotspot44162', 'hotspot44195', 'hotspot44203', 'hotspot44242', 'hotspot4429', 'hotspot44476', 'hotspot44564', 'hotspot44575', 'hotspot44613', 'hotspot44626', 'hotspot44646', 'hotspot44662', 'hotspot44839', 'hotspot44941', 'hotspot45274', 'hotspot45294', 'hotspot45364', 'hotspot45388', 'hotspot46027', 'hotspot46071', 'hotspot46138', 'hotspot46193', 'hotspot46345', 'hotspot46360', 'hotspot46623', 'hotspot46790', 'hotspot47053', 'hotspot47096', 'hotspot47417', 'hotspot47713', 'hotspot47716', 'hotspot47737', 'hotspot47880', 'hotspot47926', 'hotspot47928', 'hotspot48032', 'hotspot48176', 'hotspot48270', 'hotspot48310', 'hotspot48347', 'hotspot48417', 'hotspot48616', 'hotspot48686', 'hotspot48734', 'hotspot48742', 'hotspot48897', 'hotspot48901', 'hotspot49052', 'hotspot49134', 'hotspot49223', 'hotspot49283', 'hotspot49418', 'hotspot49548', 'hotspot49555', 'hotspot49865', 'hotspot49961', 'hotspot500', 'hotspot5006', 'hotspot50060', 'hotspot50104', 'hotspot50202', 'hotspot50263', 'hotspot50336', 'hotspot50354', 'hotspot50421', 'hotspot50432', 'hotspot50681', 'hotspot50689', 'hotspot50731', 'hotspot50752', 'hotspot51095', 'hotspot51364', 'hotspot51602', 'hotspot51623', 'hotspot51745', 'hotspot5189', 'hotspot51909', 'hotspot51943', 'hotspot52026', 'hotspot52075', 'hotspot52332', 'hotspot52762', 'hotspot52769', 'hotspot52795', 'hotspot53175', 'hotspot53336', 'hotspot53465', 'hotspot5348', 'hotspot53528', 'hotspot53727', 'hotspot53782', 'hotspot53983', 'hotspot54206', 'hotspot54290', 'hotspot54357', 'hotspot54364', 'hotspot54409', 'hotspot54410', 'hotspot5442', 'hotspot54485', 'hotspot54565', 'hotspot54733', 'hotspot54765', 'hotspot54871', 'hotspot54913', 'hotspot55055', 'hotspot55084', 'hotspot55147', 'hotspot55249', 'hotspot55325', 'hotspot55344', 'hotspot55348', 'hotspot55351', 'hotspot55687', 'hotspot55955', 'hotspot560', 'hotspot5601', 'hotspot56032', 'hotspot56070', 'hotspot5638', 'hotspot56462', 'hotspot56521', 'hotspot56602', 'hotspot56659', 'hotspot57007', 'hotspot57045', 'hotspot57064', 'hotspot57110', 'hotspot57125', 'hotspot57142', 'hotspot57164', 'hotspot5721', 'hotspot5722', 'hotspot57603', 'hotspot57753', 'hotspot57756', 'hotspot57760', 'hotspot5782', 'hotspot57956', 'hotspot58276', 'hotspot58285', 'hotspot58396', 'hotspot5849', 'hotspot58583', 'hotspot58618', 'hotspot58938', 'hotspot58945', 'hotspot59024', 'hotspot59091', 'hotspot59149', 'hotspot59182', 'hotspot59183', 'hotspot5927', 'hotspot59316', 'hotspot59357', 'hotspot59586', 'hotspot59615', 'hotspot59833', 'hotspot59928', 'hotspot59989', 'hotspot59999', 'hotspot60317', 'hotspot60329', 'hotspot6037', 'hotspot60439', 'hotspot60521', 'hotspot60647', 'hotspot60668', 'hotspot60733', 'hotspot60749', 'hotspot60800', 'hotspot60960', 'hotspot61092', 'hotspot61108', 'hotspot61232', 'hotspot61270', 'hotspot61285', 'hotspot61357', 'hotspot61363', 'hotspot61814', 'hotspot61852', 'hotspot61893', 'hotspot62052', 'hotspot62143', 'hotspot62181', 'hotspot62354', 'hotspot62405', 'hotspot62451', 'hotspot62549', 'hotspot62591', 'hotspot62631', 'hotspot62706', 'hotspot62802', 'hotspot62815', 'hotspot6291', 'hotspot62917', 'hotspot63057', 'hotspot63104', 'hotspot63234', 'hotspot63384', 'hotspot63465', 'hotspot63827', 'hotspot63857', 'hotspot63893', 'hotspot63959', 'hotspot64118', 'hotspot64130', 'hotspot64186', 'hotspot64274', 'hotspot64343', 'hotspot64405', 'hotspot64434', 'hotspot64472', 'hotspot64545', 'hotspot649', 'hotspot64979', 'hotspot65067', 'hotspot65083', 'hotspot65314', 'hotspot65410', 'hotspot6548', 'hotspot65486', 'hotspot65495', 'hotspot65556', 'hotspot65584', 'hotspot6560', 'hotspot65638', 'hotspot65664', 'hotspot65676', 'hotspot65717', 'hotspot65839', 'hotspot65880', 'hotspot65916', 'hotspot66011', 'hotspot66193', 'hotspot66247', 'hotspot663', 'hotspot66379', 'hotspot66384', 'hotspot66589', 'hotspot66745', 'hotspot66857', 'hotspot66944', 'hotspot66972', 'hotspot67007', 'hotspot67054', 'hotspot67428', 'hotspot67757', 'hotspot67857', 'hotspot67927', 'hotspot68186', 'hotspot68271', 'hotspot68571', 'hotspot68762', 'hotspot69035', 'hotspot69085', 'hotspot69102', 'hotspot69189', 'hotspot6927', 'hotspot69354', 'hotspot69438', 'hotspot69450', 'hotspot69466', 'hotspot69550', 'hotspot69672', 'hotspot697', 'hotspot69773', 'hotspot69794', 'hotspot69799', 'hotspot69814', 'hotspot69840', 'hotspot6992', 'hotspot69973', 'hotspot70022', 'hotspot70124', 'hotspot70264', 'hotspot70628', 'hotspot70635', 'hotspot70791', 'hotspot70848', 'hotspot71089', 'hotspot71218', 'hotspot71689', 'hotspot71760', 'hotspot71817', 'hotspot71828', 'hotspot71888', 'hotspot71922', 'hotspot71953', 'hotspot71983', 'hotspot72003', 'hotspot72037', 'hotspot72046', 'hotspot72147', 'hotspot72162', 'hotspot72330', 'hotspot72384', 'hotspot72694', 'hotspot72809', 'hotspot72968', 'hotspot73224', 'hotspot73241', 'hotspot73242', 'hotspot73318', 'hotspot73324', 'hotspot73331', 'hotspot7336', 'hotspot7349', 'hotspot73500', 'hotspot73659', 'hotspot73707', 'hotspot73803', 'hotspot7414', 'hotspot74197', 'hotspot74202', 'hotspot74212', 'hotspot74248', 'hotspot74406', 'hotspot74468', 'hotspot74552', 'hotspot74678', 'hotspot74823', 'hotspot7487', 'hotspot74987', 'hotspot75070', 'hotspot752', 'hotspot75399', 'hotspot75459', 'hotspot7555', 'hotspot75664', 'hotspot75667', 'hotspot75884', 'hotspot76047', 'hotspot76179', 'hotspot76229', 'hotspot76256', 'hotspot76273', 'hotspot76365', 'hotspot76373', 'hotspot76420', 'hotspot76529', 'hotspot76686', 'hotspot76711', 'hotspot76735', 'hotspot76770', 'hotspot76977', 'hotspot77001', 'hotspot77008', 'hotspot77018', 'hotspot7710', 'hotspot77424', 'hotspot77435', 'hotspot77446', 'hotspot77629', 'hotspot77654', 'hotspot77775', 'hotspot77797', 'hotspot77806', 'hotspot7786', 'hotspot77970', 'hotspot77971', 'hotspot78009', 'hotspot78180', 'hotspot78194', 'hotspot78291', 'hotspot78364', 'hotspot78437', 'hotspot78559', 'hotspot78708', 'hotspot78732', 'hotspot78920', 'hotspot78983', 'hotspot79095', 'hotspot79167', 'hotspot79182', 'hotspot79296', 'hotspot79427', 'hotspot79556', 'hotspot79627', 'hotspot79815', 'hotspot7983', 'hotspot79942', 'hotspot80020', 'hotspot80130', 'hotspot8021', 'hotspot80318', 'hotspot80431', 'hotspot80453', 'hotspot80566', 'hotspot80567', 'hotspot80810', 'hotspot80811', 'hotspot81052', 'hotspot81093', 'hotspot81229', 'hotspot81289', 'hotspot8130', 'hotspot8158', 'hotspot81643', 'hotspot81779', 'hotspot81790', 'hotspot81912', 'hotspot81915', 'hotspot82316', 'hotspot82419', 'hotspot82559', 'hotspot82567', 'hotspot82656', 'hotspot82736', 'hotspot82796', 'hotspot82873', 'hotspot82885', 'hotspot82926', 'hotspot8320', 'hotspot83448', 'hotspot8356', 'hotspot83618', 'hotspot83671', 'hotspot83725', 'hotspot83798', 'hotspot83805', 'hotspot83902', 'hotspot83904', 'hotspot83913', 'hotspot83958', 'hotspot84029', 'hotspot84046', 'hotspot84080', 'hotspot8411', 'hotspot84133', 'hotspot8429', 'hotspot84332', 'hotspot84424', 'hotspot84430', 'hotspot845', 'hotspot84588', 'hotspot8462', 'hotspot84639', 'hotspot84657', 'hotspot84781', 'hotspot84805', 'hotspot84822', 'hotspot84921', 'hotspot84965', 'hotspot85150', 'hotspot85293', 'hotspot85387', 'hotspot85474', 'hotspot85537', 'hotspot8562', 'hotspot85677', 'hotspot85855', 'hotspot85910', 'hotspot86057', 'hotspot86067', 'hotspot86133', 'hotspot86184', 'hotspot8635', 'hotspot86531', 'hotspot86623', 'hotspot8666', 'hotspot86757', 'hotspot86779', 'hotspot86935', 'hotspot87022', 'hotspot87133', 'hotspot87169', 'hotspot87257', 'hotspot87443', 'hotspot8759', 'hotspot87618', 'hotspot87695', 'hotspot87951', 'hotspot88267', 'hotspot88428', 'hotspot88448', 'hotspot88479', 'hotspot88538', 'hotspot88679', 'hotspot88683', 'hotspot88777', 'hotspot88906', 'hotspot88930', 'hotspot89001', 'hotspot89118', 'hotspot89269', 'hotspot89287', 'hotspot89291', 'hotspot89681', 'hotspot89782', 'hotspot89886', 'hotspot89904', 'hotspot89956', 'hotspot90053', 'hotspot90136', 'hotspot90288', 'hotspot90325', 'hotspot90384', 'hotspot90448', 'hotspot90570', 'hotspot90663', 'hotspot90839', 'hotspot90929', 'hotspot90968', 'hotspot90980', 'hotspot91012', 'hotspot91135', 'hotspot91297', 'hotspot91392', 'hotspot91443', 'hotspot91467', 'hotspot91663', 'hotspot91770', 'hotspot91878', 'hotspot91891', 'hotspot9197', 'hotspot92186', 'hotspot92283', 'hotspot92529', 'hotspot92531', 'hotspot92567', 'hotspot92610', 'hotspot92719', 'hotspot92923', 'hotspot93056', 'hotspot93087', 'hotspot93111', 'hotspot93200', 'hotspot93234', 'hotspot93257', 'hotspot93265', 'hotspot93280', 'hotspot93346', 'hotspot93388', 'hotspot9339', 'hotspot93415', 'hotspot93468', 'hotspot93483', 'hotspot93494', 'hotspot93534', 'hotspot93807', 'hotspot93933', 'hotspot9409', 'hotspot94297', 'hotspot94482', 'hotspot94494', 'hotspot94500', 'hotspot94553', 'hotspot94554', 'hotspot94564', 'hotspot94576', 'hotspot94591', 'hotspot94650', 'hotspot94665', 'hotspot94728', 'hotspot94865', 'hotspot9521', 'hotspot95276', 'hotspot95283', 'hotspot95470', 'hotspot95474', 'hotspot95551', 'hotspot95609', 'hotspot95647', 'hotspot95651', 'hotspot95850', 'hotspot9586', 'hotspot96093', 'hotspot96133', 'hotspot96238', 'hotspot96252', 'hotspot9650', 'hotspot96555', 'hotspot96589', 'hotspot96610', 'hotspot96716', 'hotspot9708', 'hotspot97129', 'hotspot97254', 'hotspot97296', 'hotspot97446', 'hotspot9756', 'hotspot97913', 'hotspot9802', 'hotspot9803', 'hotspot98124', 'hotspot98260', 'hotspot98280', 'hotspot98306', 'hotspot98327', 'hotspot98401', 'hotspot98447', 'hotspot98601', 'hotspot98673', 'hotspot98760', 'hotspot98912', 'hotspot98995', 'hotspot9905', 'hotspot99215', 'hotspot99372', 'hotspot99536', 'hotspot9969', 'hotspot99841', 'hotspot99943', 'hotspot99949'
    ] + [i.upper() for i in [  # from mouse
    "A2m", "AA465934;AI450353", "AK021383;Prrc2c", "AK033756;Rab3il1", "AK087340;Eif3a", "AK156288;Tpd52", "AK158346;Snrpd2", "AK164737;Ell2", "AK165270;Rbm25", "AK196308;Tuba1b", "AK202516;P4hb", "AK204572;Eef1a1", "Acsl3;Utp14b", "Aldoa", "Ankrd11", "Arl6ip1", "Atp5j", "Atrx", "Bst2", "Calr", "Cap1", "Cbx7", "Ccdc155", "Ccnb1", "Ccne1", "Cenpe", "Cenpf", "Chchd10", "Chd4;Mir7045", "Cox7b", "Cox7c", "Cox8a", "Csf2rb;Mir7676-2", "Ddx21", "Dek", "Dmkn", "Dsg2", "Eif5a", "Eif5b", "Erv3", "F11r", "Fblim1", "Glrx5", "Gm12338", "Gm15772;Rpl26", "Gm1821", "Gprc5a", "Gpx1", "Gse1", "Hbb-b2", "Hdac6", "Hdgf", "Hist1h1a", "Hist1h1b", "Hist1h1c", "Hist1h1d", "Hist1h1e", "Hist1h2ae", "Hmmr", "Hsp90aa1", "Hsp90ab1", "Hsp90b1", "Hspa5", "Hspa8", "Hspe1", "Il1r2", "Isyna1", "Kif20b", "Kmt2a", "Ldha", "Ly6a", "Ly6c1", "Manf", "Mbnl1", "Mir6236", "Mir7079;Rpl13", "Mki67", "Msh6", "Naca", "Nasp", "Ncl", "Npm1", "Pdia6", "Pfn1", "Pkm", "Plekhf2", "Pnpo", "Prc1", "Prdx2", "Prmt1", "Prpf40a", "Psip1", "Psmb10", "Ptma", "Pttg1", "Rab15", "Rell1", "Rn45s", "Rpl14-ps1", "Rpl22l1", "Rpl23", "Rpl32", "Rpl37", "Rpl38", "Rpl41", "Rpl7", "Rplp0", "Rplp1", "Rps10", "Rps11", "Rps14", "Rps15", "Rps15a-ps6", "Rps18", "Rps2", "Rps20", "Rps21", "Rps25", "Rps26", "Rps27l", "Rps4l", "Rps5", "Rpsa", "Rrm2", "S100a4", "S100a8", "Slc14a1", "Slc16a3", "Slc6a12", "Smc4", "Smox", "Soat1", "Son", "Spint1", "Spns2", "Srrm2", 
#     "Ssx2ip", 
    "Tac2", "Tjp2", "Tmem14c", "Top1", "Top2a", "Tpr", "Tubb5", "Ubb", "Ube2c", "Ung", "Hspb1", "Hspb8",
    
#     prev 2203
    'ABHD6', 'ABLIM3', 'ACI29_other_m', 'ACOT8', 'ACRA1_other_m', 'ACTE1_mm9_m', 'ADAMTSL4', 'AHSP', 'ALDC_other_m', 'ALDOA', 'ANKRD11', 'ARHGAP27P1-BPTFP1-KPNA2P3_hg19', 'ARL6IP1', 'ASIC1', 'ASPM', 'ATF7IP', 'ATP5J', 'ATP5MC3_hg19', 'ATRX', 'AURKA', 'AURKB', 'BCLAF1', 'BEX3', 'BNIP3', 'BOD1L1', 'BPTF', 'BRD4', 'BUB1', 'BUB1B', 'C12H17ORF75_other_m', 'C12H6orf229', 'C1H11orf31', 'CALR', 'CAP1', 'CBX5', 'CBX7', 'CCDC155', 'CCL27', 'CCNB1', 'CCNB2', 'CCND2', 'CCNE1', 'CD8B', 'CDC20;MPL', 'CDCA2', 'CDCA3', 'CDCA7', 'CDK1', 'CDK3', 'CENPC', 'CENPE', 'CENPF', 'CENPV', 'CEP128', 'CHCHD10', 'CHD4', 'CHD7', 'CHST1', 'CKAP2L', 'CKAP5', 'CKS1_other_m', 'CLEC2D_hg19', 'CLSPN', 'COL6A5', 'COQ2', 'COQ3;PNISR', 'COX15_mm9', 'COX16_mm9_m', 'COX20_mm9', 'COX4I2_other_m', 'COX6A1', 'COX6C_mm9', 'COX7A2L_hg19', 'COX7A2_mm9', 'COX7B_mm9_m', 'COX7C_mm9_m', 'CPN10_other', 'CROT', 'CTTNBP2', 'DDX21', 'DEK', 'DEPDC1B', 'DGUOK', 'DLGAP5', 'DMKN', 'DSG2', 'DTL', 'DUSP13', 'ECT2', 'EEF1A1', 'EEF1B2', 'EFCAB5', 'EGR1', 'EIF3A', 'EIF4G1', 'EIF4G2', 'EIF5A', 'EIF5B', 'ELOVL4', 'ENO1', 'F11R', 'F2RL1', 'FADS1', 'FAM110C', 'FAM162A', 'FAM60A', 'FAM64A', 'FAT1', 'FERMT1', 'FLJ44635_hg19', 'FNDC10_hg19', 'FOS', 'GADD45A', 'GFPT2', 'GLRX5', 'GM15501_mm9', 'GM4925_mm9', 'GM5069_mm9_m', 'GM5176_mm9', 'GM6607_mm9', 'GNAI2', 'GPRC5A', 'GPX1', 'GRB14', 'GSTP1', 'HBB2', 'HBG1', 'HBG2', 'HBZ', 'HDAC2_mm9', 'HDAC6', 'HDGF', 'HIGD1A', 'HIVEP2', 'HMMR', 'HS1BP3', 'HS2ST1', 'HS3ST1', 'HS3ST2', 'HS3ST3A1', 'HS3ST3B1', 'HS3ST3L_other', 'HS3ST4', 'HS3ST5', 'HS3ST6', 'HS6ST2', 'HS6ST3', 'HSBP1L1', 'HSCB', 'HSD11B1', 'HSD11B1L', 'HSD11B2', 'HSD17B1', 'HSD17B10', 'HSD17B12', 'HSD17B2', 'HSD17B3', 'HSD17B4', 'HSD17B7', 'HSD17B8', 'HSD3B7', 'HSDL1', 'HSDL2', 'HSF1', 'HSF2', 'HSF2BP', 'HSF4', 'HSF5', 'HSH2D', 'HSP20.1_other', 'HSP90AA1', 'HSP90AB1', 'HSP90B1', 'HSPA12A', 'HSPA12B', 'HSPA13', 'HSPA14', 'HSPA1A.S_other_m', 'HSPA1B_mm9', 'HSPA2', 'HSPA4', 'HSPA4L', 'HSPA5', 'HSPA6', 'HSPA8', 'HSPA9', 'HSPB1', 'HSPB11', 'HSPB2', 'HSPB6', 'HSPB7', 'HSPB8', 'HSPB9', 'HSPBAP1', 'HSPBP1', 'HSPD1', 'HSPD1.S_other', 'HSPE1', 'HSPG2', 'HSPH1', 'HUWE1', 'HYLS1', 'HYOU1', 'IFI35', 'IFI44', 'IFI44L', 'IFIH1', 'IFIT2', 'IFIT3', 'IFIT5', 'IFITM10_mm9', 'IFITM1_mm9', 'IGHMBP2', 'IGLL1', 'IL1R2', 'IRF7', 'ISG15', 'K11H12.1_other_m', 'KCTD7', 'KDM5B', 'KHDRBS2', 'KIF11', 'KIF14', 'KIF18A', 'KIF20A', 'KIF20B', 'KIF4A', 'KIFC1', 'KLRG2', 'KMT2A', 'KMT2C', 'KNL1', 'KNSTRN', 'KTN1', 'LARP7', 'LDHA', 'LDHC_mm9_m', 'LIN28_other_m', 'LOC100287467', 'LOC100313593', 'LOC100338112', 'LOC100339133', 'LOC100339314', 'LOC100340645', 'LOC100341037', 'LOC100341584', 'LOC100341713', 'LOC100342501', 'LOC100342635', 'LOC100343039', 'LOC100343268', 'LOC100343557', 'LOC100343802', 'LOC100343982', 'LOC100344007', 'LOC100344156', 'LOC100344388', 'LOC100345882', 'LOC100345978', 'LOC100346597', 'LOC100348336', 'LOC100348744', 'LOC100348796', 'LOC100349084', 'LOC100349527', 'LOC100351037', 'LOC100351127', 'LOC100351519', 'LOC100353361', 'LOC100353654;LOC108176166', 'LOC100353836', 'LOC100356667', 'LOC100357307', 'LOC100357840', 'LOC100357913', 'LOC100358241', 'LOC100358336', 'LOC102723302', 'LOC103345230', 'LOC103345596', 'LOC103346294', 'LOC103346930', 'LOC103347883', 'LOC103349265', 'LOC103349305', 'LOC103349783', 'LOC103969081', 'LOC106105278;LOC106116970', 'LOC108176098', 'LOC108176430', 'LOC108176625', 'LOC108176694', 'LOC108176887', 'LOC108177422', 'LOC108178055', 'LOC108178549', 'LOC108178646', 'MAP3K5', 'MARCKSL1', 'MAVS', 'MBNL1', 'MCM10', 'MCM2', 'MCM3', 'MCM3AP', 'MCM4', 'MCM5', 'MCM6', 'MCM7', 'MCM8', 'MCM9', 'MFSD8', 'MGA', 'MIF', 'MIR682_mm9', 'MIS18BP1', 'MKI67', 'MMD', 'MPP7', 'MSH6', 'MYO1F', 'NACA', 'NASP', 'NCAPD2', 'NCAPG', 'NCL', 'NCL.L_other_m', 'NDC80', 'NEFM', 'NEK2', 'NKIRAS1', 'NKTR', 'NOP56', 'NOS3', 'NPM1', 'NPM3', 'NRL', 'NSD1', 'NUCKS1', 'NUSAP1', 'OLFR645', 'PARP1', 'PBLD', 'PCLO', 'PCNA', 'PDE6D', 'PDIA6', 'PFKP', 'PFN1', 'PGAM1', 'PGAM1P5_hg19_m', 'PGF', 'PGK1', 'PIF1', 'PKM', 'PLEKHF2', 'PLK1', 'PLXDC1', 'PNPO', 'POR', 'PPARD', 'PPIA', 'PPIAL4A_hg19_m', 'PPM1J', 'PPP1R12B', 'PRC1', 'PRDX1', 'PRDX6', 'PRMT1', 'PRPF40A', 'PRR11', 'PRRC2C', 'PSIP1', 'PSMB10', 'PSRC1', 'PTMA', 'PTTG1', 'RAB15', 'RBBP6', 'RELL1', 'RFX4', 'RIF1', 'RPL10', 'RPL11', 'RPL17_mm9_m', 'RPL18A', 'RPL19', 'RPL19P12_hg19', 'RPL21', 'RPL22L1', 'RPL23', 'RPL23B_other', 'RPL27A', 'RPL31', 'RPL31_other_m', 'RPL32', 'RPL37', 'RPL41_mm9', 'RPL7', 'RPL8', 'RPLP0', 'RPLP1', 'RPS-24_other', 'RPS10', 'RPS11_other', 'RPS12', 'RPS14', 'RPS18', 'RPS20', 'RPS21', 'RPS25', 'RPS26', 'RPS27L', 'RPS5', 'RPS6-PS4_mm9', 'RPSA', 'RRM2', 'RS5-8S1_other', 'RSF1', 'RSL1D1', 'S100A8', 'SESN2', 'SGO2', 'SLC16A3', 'SLC25A5', 'SLC25A6_hg19', 'SLC39A5', 'SLC6A12', 'SLC7A1', 'SMARCC1', 'SMC1A', 'SMC1B', 'SMC2', 'SMC3', 'SMC4', 'SMC4.L_other', 'SMC5', 'SMC6', 'SMOX', 'SNORA36B_mm9_m', 'SOAT1', 'SON', 'SPAG5', 'SPINT1', 'SPNS2', 'SREK1', 'SRRM2', 'SRSF2', 'ST8SIA1', 'STIM1', 'SYN2', 'SYTL4', 'TARDBPP3_hg19', 'THRAP3', 'TJP2', 'TMEM14C', 'TMPRSS11D', 'TNFRSF21', 'TNRC6B', 'TOP1', 'TOP2A', 'TP53', 'TPI1', 'TPR', 'TPT1', 'TPX2', 'TSC1', 'TTK', 'UAP1', 'UBASH3B', 'UBB', 'UBE2C', 'UHMK1', 'UNC80', 'UNG', 'WDR35', 'YBX3', 'ZC3H13', 'ZFP36L2', 'ZFP64', 'ZNF704', 'hotspot100013', 'hotspot100015', 'hotspot100056', 'hotspot100078', 'hotspot100079', 'hotspot100118', 'hotspot100369', 'hotspot100383', 'hotspot100432', 'hotspot10048', 'hotspot1005', 'hotspot100610', 'hotspot100646', 'hotspot100684', 'hotspot100815', 'hotspot100884', 'hotspot100909', 'hotspot100929', 'hotspot100941', 'hotspot101000', 'hotspot101082', 'hotspot101087', 'hotspot101135', 'hotspot101221', 'hotspot101241', 'hotspot101363', 'hotspot101596', 'hotspot101658', 'hotspot101711', 'hotspot101850', 'hotspot101986', 'hotspot102177', 'hotspot102239', 'hotspot102344', 'hotspot102506', 'hotspot10265', 'hotspot102652', 'hotspot102693', 'hotspot102708', 'hotspot102714', 'hotspot10272', 'hotspot102738', 'hotspot102755', 'hotspot10285', 'hotspot102901', 'hotspot102996', 'hotspot103208', 'hotspot103428', 'hotspot103459', 'hotspot103536', 'hotspot103597', 'hotspot103612', 'hotspot103627', 'hotspot103659', 'hotspot103724', 'hotspot103725', 'hotspot103749', 'hotspot103759', 'hotspot103784', 'hotspot103812', 'hotspot103816', 'hotspot103840', 'hotspot103885', 'hotspot104110', 'hotspot10418', 'hotspot104184', 'hotspot104274', 'hotspot104296', 'hotspot104300', 'hotspot104308', 'hotspot104317', 'hotspot104355', 'hotspot10439', 'hotspot104393', 'hotspot104499', 'hotspot10455', 'hotspot104580', 'hotspot104582', 'hotspot104604', 'hotspot104639', 'hotspot104792', 'hotspot1048', 'hotspot104902', 'hotspot104918', 'hotspot105014', 'hotspot105093', 'hotspot10512', 'hotspot105121', 'hotspot105180', 'hotspot105207', 'hotspot10523', 'hotspot105248', 'hotspot105296', 'hotspot105343', 'hotspot10541', 'hotspot105482', 'hotspot105601', 'hotspot10561', 'hotspot105657', 'hotspot105694', 'hotspot105706', 'hotspot105801', 'hotspot105808', 'hotspot105821', 'hotspot10589', 'hotspot105988', 'hotspot106057', 'hotspot106060', 'hotspot106082', 'hotspot106084', 'hotspot106231', 'hotspot106292', 'hotspot106397', 'hotspot106573', 'hotspot106632', 'hotspot106664', 'hotspot10681', 'hotspot106836', 'hotspot10684', 'hotspot10685', 'hotspot10686', 'hotspot106889', 'hotspot106931', 'hotspot106947', 'hotspot10695', 'hotspot106993', 'hotspot107093', 'hotspot107284', 'hotspot107341', 'hotspot107412', 'hotspot107508', 'hotspot107513', 'hotspot10758', 'hotspot10764', 'hotspot107741', 'hotspot107812', 'hotspot107906', 'hotspot108196', 'hotspot108321', 'hotspot108374', 'hotspot108405', 'hotspot108604', 'hotspot108624', 'hotspot10864', 'hotspot108797', 'hotspot108807', 'hotspot108837', 'hotspot108908', 'hotspot108909', 'hotspot109043', 'hotspot109109', 'hotspot109285', 'hotspot109336', 'hotspot109392', 'hotspot109580', 'hotspot109585', 'hotspot109644', 'hotspot109714', 'hotspot109810', 'hotspot109817', 'hotspot109946', 'hotspot109968', 'hotspot110022', 'hotspot110031', 'hotspot110077', 'hotspot110083', 'hotspot110117', 'hotspot110250', 'hotspot11038', 'hotspot110426', 'hotspot110561', 'hotspot110580', 'hotspot110582', 'hotspot110583', 'hotspot110602', 'hotspot110707', 'hotspot110741', 'hotspot111178', 'hotspot111188', 'hotspot11122', 'hotspot111371', 'hotspot111411', 'hotspot111532', 'hotspot111556', 'hotspot11160', 'hotspot111635', 'hotspot111652', 'hotspot111716', 'hotspot111727', 'hotspot111735', 'hotspot111737', 'hotspot111955', 'hotspot111980', 'hotspot112166', 'hotspot112237', 'hotspot112370', 'hotspot112372', 'hotspot11239', 'hotspot112632', 'hotspot112739', 'hotspot112760', 'hotspot112784', 'hotspot112806', 'hotspot112809', 'hotspot11283', 'hotspot113009', 'hotspot113063', 'hotspot113114', 'hotspot113158', 'hotspot113176', 'hotspot113188', 'hotspot113190', 'hotspot113196', 'hotspot113203', 'hotspot113225', 'hotspot113330', 'hotspot1134', 'hotspot113406', 'hotspot113441', 'hotspot113442', 'hotspot113582', 'hotspot113651', 'hotspot113683', 'hotspot113714', 'hotspot113815', 'hotspot113899', 'hotspot113949', 'hotspot113972', 'hotspot114024', 'hotspot114053', 'hotspot114163', 'hotspot114261', 'hotspot114276', 'hotspot114395', 'hotspot114460', 'hotspot114465', 'hotspot114548', 'hotspot114606', 'hotspot114661', 'hotspot114677', 'hotspot114824', 'hotspot11483', 'hotspot114833', 'hotspot11496', 'hotspot114972', 'hotspot115076', 'hotspot115156', 'hotspot115222', 'hotspot115277', 'hotspot115312', 'hotspot115428', 'hotspot115459', 'hotspot115460', 'hotspot115471', 'hotspot115493', 'hotspot115655', 'hotspot115745', 'hotspot115751', 'hotspot115773', 'hotspot115832', 'hotspot115844', 'hotspot115893', 'hotspot115931', 'hotspot115956', 'hotspot115978', 'hotspot116210', 'hotspot116271', 'hotspot116291', 'hotspot116493', 'hotspot11652', 'hotspot116551', 'hotspot116563', 'hotspot116596', 'hotspot116665', 'hotspot116670', 'hotspot116699', 'hotspot116713', 'hotspot116740', 'hotspot116752', 'hotspot116786', 'hotspot116859', 'hotspot117009', 'hotspot117096', 'hotspot117162', 'hotspot117165', 'hotspot117201', 'hotspot117214', 'hotspot117275', 'hotspot1173', 'hotspot11749', 'hotspot11793', 'hotspot12024', 'hotspot12077', 'hotspot12088', 'hotspot121', 'hotspot12171', 'hotspot12192', 'hotspot12264', 'hotspot12270', 'hotspot12275', 'hotspot12310', 'hotspot12353', 'hotspot12480', 'hotspot12494', 'hotspot12634', 'hotspot12697', 'hotspot12724', 'hotspot12820', 'hotspot12854', 'hotspot12930', 'hotspot12956', 'hotspot12972', 'hotspot1298', 'hotspot13030', 'hotspot13075', 'hotspot13128', 'hotspot13178', 'hotspot13398', 'hotspot13401', 'hotspot13422', 'hotspot13495', 'hotspot13548', 'hotspot13742', 'hotspot13840', 'hotspot13927', 'hotspot13938', 'hotspot13965', 'hotspot1402', 'hotspot14030', 'hotspot14041', 'hotspot14044', 'hotspot1406', 'hotspot14197', 'hotspot14199', 'hotspot14246', 'hotspot14248', 'hotspot14271', 'hotspot14371', 'hotspot14417', 'hotspot14584', 'hotspot14624', 'hotspot14740', 'hotspot14839', 'hotspot1498', 'hotspot15389', 'hotspot15406', 'hotspot15480', 'hotspot15520', 'hotspot15556', 'hotspot15638', 'hotspot15769', 'hotspot15909', 'hotspot15986', 'hotspot15995', 'hotspot16156', 'hotspot1630', 'hotspot16388', 'hotspot16398', 'hotspot16516', 'hotspot16560', 'hotspot16745', 'hotspot1677', 'hotspot16856', 'hotspot16944', 'hotspot17094', 'hotspot17171', 'hotspot17203', 'hotspot17260', 'hotspot17320', 'hotspot17351', 'hotspot17402', 'hotspot17417', 'hotspot17439', 'hotspot17489', 'hotspot17528', 'hotspot17595', 'hotspot17606', 'hotspot17610', 'hotspot17750', 'hotspot17771', 'hotspot17822', 'hotspot17824', 'hotspot18115', 'hotspot18204', 'hotspot18272', 'hotspot18288', 'hotspot18361', 'hotspot18382', 'hotspot18411', 'hotspot18461', 'hotspot1848', 'hotspot18627', 'hotspot18814', 'hotspot18974', 'hotspot18997', 'hotspot19077', 'hotspot19095', 'hotspot19186', 'hotspot19289', 'hotspot19361', 'hotspot1940', 'hotspot19513', 'hotspot19546', 'hotspot1960', 'hotspot19662', 'hotspot19688', 'hotspot19705', 'hotspot19717', 'hotspot19811', 'hotspot19834', 'hotspot19916', 'hotspot19960', 'hotspot19982', 'hotspot19988', 'hotspot20063', 'hotspot20137', 'hotspot20173', 'hotspot20386', 'hotspot20391', 'hotspot204', 'hotspot20471', 'hotspot20555', 'hotspot20646', 'hotspot20647', 'hotspot20685', 'hotspot2075', 'hotspot20753', 'hotspot2078', 'hotspot20940', 'hotspot21022', 'hotspot21034', 'hotspot21107', 'hotspot21194', 'hotspot212', 'hotspot21559', 'hotspot21579', 'hotspot21622', 'hotspot21683', 'hotspot2169', 'hotspot21736', 'hotspot21889', 'hotspot21897', 'hotspot21904', 'hotspot2226', 'hotspot22265', 'hotspot22269', 'hotspot2231', 'hotspot22409', 'hotspot22514', 'hotspot22535', 'hotspot22579', 'hotspot22625', 'hotspot22689', 'hotspot22797', 'hotspot22955', 'hotspot2299', 'hotspot23037', 'hotspot2305', 'hotspot23084', 'hotspot23109', 'hotspot23121', 'hotspot2317', 'hotspot23204', 'hotspot23230', 'hotspot23255', 'hotspot23273', 'hotspot23297', 'hotspot233', 'hotspot23379', 'hotspot23405', 'hotspot23480', 'hotspot23542', 'hotspot23585', 'hotspot23736', 'hotspot23792', 'hotspot23888', 'hotspot23923', 'hotspot24033', 'hotspot24169', 'hotspot24224', 'hotspot24229', 'hotspot24316', 'hotspot24351', 'hotspot24372', 'hotspot24435', 'hotspot24438', 'hotspot24489', 'hotspot24493', 'hotspot24528', 'hotspot24641', 'hotspot24706', 'hotspot24767', 'hotspot24798', 'hotspot2489', 'hotspot24937', 'hotspot24962', 'hotspot25059', 'hotspot25060', 'hotspot25066', 'hotspot25110', 'hotspot25111', 'hotspot25151', 'hotspot252', 'hotspot25247', 'hotspot25268', 'hotspot25304', 'hotspot25313', 'hotspot25381', 'hotspot25511', 'hotspot25553', 'hotspot25646', 'hotspot25854', 'hotspot2586', 'hotspot25897', 'hotspot2604', 'hotspot26133', 'hotspot26325', 'hotspot26442', 'hotspot26524', 'hotspot26686', 'hotspot26688', 'hotspot26776', 'hotspot26810', 'hotspot26855', 'hotspot27014', 'hotspot27027', 'hotspot2732', 'hotspot27334', 'hotspot27367', 'hotspot27448', 'hotspot27509', 'hotspot27594', 'hotspot27708', 'hotspot27766', 'hotspot2781', 'hotspot27891', 'hotspot28057', 'hotspot28137', 'hotspot28144', 'hotspot28163', 'hotspot2822', 'hotspot28355', 'hotspot28382', 'hotspot28520', 'hotspot28551', 'hotspot28594', 'hotspot28787', 'hotspot28816', 'hotspot28886', 'hotspot28910', 'hotspot28924', 'hotspot28942', 'hotspot29108', 'hotspot29471', 'hotspot29492', 'hotspot29716', 'hotspot29753', 'hotspot29795', 'hotspot29839', 'hotspot29846', 'hotspot30205', 'hotspot30301', 'hotspot30308', 'hotspot30320', 'hotspot30342', 'hotspot30572', 'hotspot3071', 'hotspot30898', 'hotspot30908', 'hotspot31092', 'hotspot31160', 'hotspot31213', 'hotspot31339', 'hotspot31469', 'hotspot31473', 'hotspot31530', 'hotspot31782', 'hotspot31835', 'hotspot3192', 'hotspot31930', 'hotspot31980', 'hotspot32083', 'hotspot32092', 'hotspot32176', 'hotspot32260', 'hotspot32318', 'hotspot32435', 'hotspot32530', 'hotspot326', 'hotspot32610', 'hotspot32709', 'hotspot32721', 'hotspot3274', 'hotspot32801', 'hotspot32837', 'hotspot32838', 'hotspot32864', 'hotspot3291', 'hotspot32983', 'hotspot3300', 'hotspot33009', 'hotspot33069', 'hotspot33107', 'hotspot33188', 'hotspot3322', 'hotspot33289', 'hotspot33499', 'hotspot33678', 'hotspot33713', 'hotspot3381', 'hotspot33848', 'hotspot33893', 'hotspot33895', 'hotspot33930', 'hotspot34047', 'hotspot3406', 'hotspot34177', 'hotspot34230', 'hotspot34347', 'hotspot34425', 'hotspot34450', 'hotspot34465', 'hotspot34515', 'hotspot34530', 'hotspot34542', 'hotspot34564', 'hotspot3468', 'hotspot34714', 'hotspot34870', 'hotspot34943', 'hotspot35215', 'hotspot35548', 'hotspot3557', 'hotspot35602', 'hotspot35793', 'hotspot35887', 'hotspot36274', 'hotspot36381', 'hotspot3646', 'hotspot36477', 'hotspot365', 'hotspot36538', 'hotspot36629', 'hotspot36706', 'hotspot36709', 'hotspot36739', 'hotspot3681', 'hotspot36841', 'hotspot36908', 'hotspot37', 'hotspot37004', 'hotspot37020', 'hotspot37047', 'hotspot37111', 'hotspot37262', 'hotspot37294', 'hotspot37376', 'hotspot37382', 'hotspot37411', 'hotspot37448', 'hotspot37589', 'hotspot37630', 'hotspot37642', 'hotspot37664', 'hotspot37807', 'hotspot37857', 'hotspot37958', 'hotspot38087', 'hotspot38089', 'hotspot38090', 'hotspot38279', 'hotspot38333', 'hotspot38409', 'hotspot38410', 'hotspot38497', 'hotspot38569', 'hotspot39107', 'hotspot3916', 'hotspot39196', 'hotspot39476', 'hotspot39495', 'hotspot39571', 'hotspot39700', 'hotspot39739', 'hotspot3975', 'hotspot39882', 'hotspot39920', 'hotspot40116', 'hotspot40150', 'hotspot40158', 'hotspot40259', 'hotspot40282', 'hotspot40288', 'hotspot40361', 'hotspot40395', 'hotspot40504', 'hotspot40625', 'hotspot40913', 'hotspot41142', 'hotspot41339', 'hotspot41415', 'hotspot41505', 'hotspot4152', 'hotspot41628', 'hotspot41799', 'hotspot41804', 'hotspot41907', 'hotspot41970', 'hotspot42010', 'hotspot42022', 'hotspot42081', 'hotspot42121', 'hotspot42123', 'hotspot42139', 'hotspot42146', 'hotspot42499', 'hotspot42512', 'hotspot42754', 'hotspot42899', 'hotspot42964', 'hotspot4298', 'hotspot4303', 'hotspot4321', 'hotspot43243', 'hotspot43326', 'hotspot4367', 'hotspot43695', 'hotspot43761', 'hotspot43796', 'hotspot43832', 'hotspot43869', 'hotspot4391', 'hotspot43912', 'hotspot44069', 'hotspot44139', 'hotspot44162', 'hotspot44195', 'hotspot44203', 'hotspot44242', 'hotspot4429', 'hotspot44319', 'hotspot44330', 'hotspot44476', 'hotspot44564', 'hotspot44575', 'hotspot44613', 'hotspot44626', 'hotspot44646', 'hotspot44662', 'hotspot44839', 'hotspot44941', 'hotspot45006', 'hotspot45200', 'hotspot45274', 'hotspot45293', 'hotspot45294', 'hotspot45319', 'hotspot45364', 'hotspot45388', 'hotspot4572', 'hotspot45815', 'hotspot46027', 'hotspot46071', 'hotspot46138', 'hotspot46193', 'hotspot46345', 'hotspot46360', 'hotspot4637', 'hotspot4655', 'hotspot46623', 'hotspot46790', 'hotspot46943', 'hotspot47053', 'hotspot47096', 'hotspot47417', 'hotspot47560', 'hotspot47713', 'hotspot47716', 'hotspot47718', 'hotspot47737', 'hotspot47806', 'hotspot47880', 'hotspot47926', 'hotspot47928', 'hotspot48032', 'hotspot4811', 'hotspot48176', 'hotspot48270', 'hotspot48310', 'hotspot48328', 'hotspot48347', 'hotspot48417', 'hotspot48616', 'hotspot48686', 'hotspot48734', 'hotspot48742', 'hotspot48897', 'hotspot48901', 'hotspot49052', 'hotspot49134', 'hotspot49223', 'hotspot49283', 'hotspot49378', 'hotspot49418', 'hotspot49462', 'hotspot49548', 'hotspot49555', 'hotspot49675', 'hotspot49865', 'hotspot49961', 'hotspot500', 'hotspot50014', 'hotspot5006', 'hotspot50060', 'hotspot50104', 'hotspot50202', 'hotspot50244', 'hotspot50263', 'hotspot50336', 'hotspot50354', 'hotspot50421', 'hotspot50432', 'hotspot50552', 'hotspot50651', 'hotspot50681', 'hotspot50689', 'hotspot50731', 'hotspot50745', 'hotspot50752', 'hotspot51095', 'hotspot51364', 'hotspot51602', 'hotspot51623', 'hotspot51745', 'hotspot51860', 'hotspot5189', 'hotspot51909', 'hotspot51943', 'hotspot52026', 'hotspot52050', 'hotspot52075', 'hotspot52137', 'hotspot52178', 'hotspot52273', 'hotspot52278', 'hotspot52332', 'hotspot52762', 'hotspot52769', 'hotspot52795', 'hotspot53175', 'hotspot53227', 'hotspot53266', 'hotspot53336', 'hotspot53465', 'hotspot5348', 'hotspot53528', 'hotspot53727', 'hotspot53782', 'hotspot53983', 'hotspot54206', 'hotspot54290', 'hotspot54357', 'hotspot54364', 'hotspot54409', 'hotspot54410', 'hotspot5442', 'hotspot54485', 'hotspot54565', 'hotspot54733', 'hotspot54765', 'hotspot54820', 'hotspot54871', 'hotspot54913', 'hotspot55055', 'hotspot55084', 'hotspot55147', 'hotspot55249', 'hotspot55325', 'hotspot55344', 'hotspot55348', 'hotspot55351', 'hotspot55687', 'hotspot55931', 'hotspot55955', 'hotspot560', 'hotspot5601', 'hotspot56032', 'hotspot56070', 'hotspot56098', 'hotspot56367', 'hotspot5638', 'hotspot56448', 'hotspot56462', 'hotspot56521', 'hotspot56602', 'hotspot56659', 'hotspot56924', 'hotspot57007', 'hotspot57045', 'hotspot57064', 'hotspot5709', 'hotspot57110', 'hotspot57125', 'hotspot57142', 'hotspot57164', 'hotspot5721', 'hotspot5722', 'hotspot57456', 'hotspot57603', 'hotspot57729', 'hotspot57753', 'hotspot57756', 'hotspot57758', 'hotspot57760', 'hotspot57807', 'hotspot5782', 'hotspot57892', 'hotspot5794', 'hotspot57956', 'hotspot5802', 'hotspot58105', 'hotspot58276', 'hotspot58285', 'hotspot58396', 'hotspot58434', 'hotspot5849', 'hotspot58530', 'hotspot58583', 'hotspot58618', 'hotspot58705', 'hotspot58938', 'hotspot58945', 'hotspot59024', 'hotspot59082', 'hotspot59091', 'hotspot59149', 'hotspot59160', 'hotspot59182', 'hotspot59183', 'hotspot59248', 'hotspot5927', 'hotspot59316', 'hotspot59357', 'hotspot59586', 'hotspot59615', 'hotspot59616', 'hotspot59684', 'hotspot59833', 'hotspot59834', 'hotspot59928', 'hotspot59989', 'hotspot59999', 'hotspot60149', 'hotspot60317', 'hotspot60329', 'hotspot6037', 'hotspot60406', 'hotspot60439', 'hotspot60521', 'hotspot60647', 'hotspot60668', 'hotspot60733', 'hotspot60749', 'hotspot60800', 'hotspot6094', 'hotspot60960', 'hotspot61092', 'hotspot61108', 'hotspot61232', 'hotspot61270', 'hotspot61285', 'hotspot61357', 'hotspot61363', 'hotspot61600', 'hotspot61814', 'hotspot61852', 'hotspot61873', 'hotspot61893', 'hotspot62052', 'hotspot62143', 'hotspot62181', 'hotspot62354', 'hotspot62405', 'hotspot62451', 'hotspot6248', 'hotspot62549', 'hotspot62591', 'hotspot62631', 'hotspot62706', 'hotspot62802', 'hotspot62815', 'hotspot62871', 'hotspot6291', 'hotspot62917', 'hotspot63057', 'hotspot63104', 'hotspot63234', 'hotspot63384', 'hotspot63465', 'hotspot63714', 'hotspot63827', 'hotspot63857', 'hotspot63893', 'hotspot63934', 'hotspot63959', 'hotspot63967', 'hotspot64118', 'hotspot64130', 'hotspot64186', 'hotspot64274', 'hotspot64343', 'hotspot64405', 'hotspot64434', 'hotspot64472', 'hotspot64545', 'hotspot64680', 'hotspot649', 'hotspot64979', 'hotspot65067', 'hotspot65083', 'hotspot65120', 'hotspot65314', 'hotspot65315', 'hotspot6536', 'hotspot65404', 'hotspot65410', 'hotspot6548', 'hotspot65486', 'hotspot65495', 'hotspot655', 'hotspot65553', 'hotspot65556', 'hotspot65584', 'hotspot6560', 'hotspot65638', 'hotspot65664', 'hotspot65676', 'hotspot65717', 'hotspot65839', 'hotspot65842', 'hotspot65880', 'hotspot65916', 'hotspot66011', 'hotspot66193', 'hotspot66216', 'hotspot66247', 'hotspot663', 'hotspot66379', 'hotspot66384', 'hotspot66589', 'hotspot66745', 'hotspot66839', 'hotspot66857', 'hotspot66944', 'hotspot66972', 'hotspot67007', 'hotspot67054', 'hotspot6735', 'hotspot67428', 'hotspot6767', 'hotspot67757', 'hotspot67857', 'hotspot67910', 'hotspot67927', 'hotspot68117', 'hotspot68186', 'hotspot68271', 'hotspot68571', 'hotspot6867', 'hotspot68762', 'hotspot69035', 'hotspot69041', 'hotspot69085', 'hotspot69102', 'hotspot69189', 'hotspot69258', 'hotspot6927', 'hotspot69304', 'hotspot69354', 'hotspot69438', 'hotspot69450', 'hotspot69466', 'hotspot69550', 'hotspot69672', 'hotspot69686', 'hotspot697', 'hotspot69773', 'hotspot69794', 'hotspot69799', 'hotspot69814', 'hotspot69840', 'hotspot6992', 'hotspot69972', 'hotspot69973', 'hotspot70022', 'hotspot70124', 'hotspot70264', 'hotspot70545', 'hotspot70628', 'hotspot70635', 'hotspot70689', 'hotspot70791', 'hotspot70848', 'hotspot71089', 'hotspot71218', 'hotspot71262', 'hotspot71502', 'hotspot71518', 'hotspot71542', 'hotspot71689', 'hotspot71691', 'hotspot71760', 'hotspot71817', 'hotspot71828', 'hotspot71888', 'hotspot71922', 'hotspot71928', 'hotspot71953', 'hotspot71983', 'hotspot72003', 'hotspot72037', 'hotspot72046', 'hotspot72108', 'hotspot72147', 'hotspot72162', 'hotspot72330', 'hotspot72384', 'hotspot72449', 'hotspot72694', 'hotspot72791', 'hotspot72809', 'hotspot72901', 'hotspot72923', 'hotspot72968', 'hotspot73224', 'hotspot73226', 'hotspot73241', 'hotspot73242', 'hotspot73318', 'hotspot73324', 'hotspot73331', 'hotspot7336', 'hotspot7349', 'hotspot73500', 'hotspot73584', 'hotspot73593', 'hotspot73659', 'hotspot73707', 'hotspot73803', 'hotspot7414', 'hotspot74197', 'hotspot74202', 'hotspot74212', 'hotspot74248', 'hotspot74406', 'hotspot74468', 'hotspot74552', 'hotspot74678', 'hotspot74823', 'hotspot7487', 'hotspot7493', 'hotspot74987', 'hotspot75035', 'hotspot75061', 'hotspot75070', 'hotspot752', 'hotspot75328', 'hotspot75399', 'hotspot75459', 'hotspot7555', 'hotspot75664', 'hotspot75667', 'hotspot75884', 'hotspot76047', 'hotspot76179', 'hotspot76229', 'hotspot76246', 'hotspot76256', 'hotspot76273', 'hotspot76282', 'hotspot76283', 'hotspot76365', 'hotspot76373', 'hotspot76420', 'hotspot76529', 'hotspot76665', 'hotspot76686', 'hotspot76711', 'hotspot76735', 'hotspot76743', 'hotspot76770', 'hotspot7678', 'hotspot76977', 'hotspot77001', 'hotspot77008', 'hotspot77018', 'hotspot7710', 'hotspot77223', 'hotspot77424', 'hotspot77435', 'hotspot77446', 'hotspot7756', 'hotspot77613', 'hotspot77629', 'hotspot77654', 'hotspot77775', 'hotspot77797', 'hotspot77806', 'hotspot7786', 'hotspot77970', 'hotspot77971', 'hotspot78009', 'hotspot78180', 'hotspot78194', 'hotspot78232', 'hotspot78291', 'hotspot78314', 'hotspot78364', 'hotspot78437', 'hotspot78559', 'hotspot78708', 'hotspot78732', 'hotspot78775', 'hotspot78920', 'hotspot78983', 'hotspot79095', 'hotspot79167', 'hotspot79182', 'hotspot79208', 'hotspot79296', 'hotspot79427', 'hotspot79556', 'hotspot79562', 'hotspot79604', 'hotspot79627', 'hotspot79815', 'hotspot7983', 'hotspot79942', 'hotspot80020', 'hotspot80130', 'hotspot8021', 'hotspot8029', 'hotspot80318', 'hotspot80431', 'hotspot80453', 'hotspot80566', 'hotspot80567', 'hotspot80709', 'hotspot80810', 'hotspot80811', 'hotspot80932', 'hotspot81052', 'hotspot81093', 'hotspot81203', 'hotspot81229', 'hotspot81289', 'hotspot8130', 'hotspot8138', 'hotspot81527', 'hotspot8158', 'hotspot81643', 'hotspot81779', 'hotspot81790', 'hotspot81912', 'hotspot81915', 'hotspot82013', 'hotspot82237', 'hotspot82316', 'hotspot82419', 'hotspot82559', 'hotspot82567', 'hotspot82597', 'hotspot82656', 'hotspot82736', 'hotspot82756', 'hotspot82790', 'hotspot82796', 'hotspot82873', 'hotspot82885', 'hotspot82926', 'hotspot8320', 'hotspot83448', 'hotspot8356', 'hotspot83618', 'hotspot83671', 'hotspot83725', 'hotspot83798', 'hotspot83805', 'hotspot83902', 'hotspot83904', 'hotspot83905', 'hotspot83913', 'hotspot83958', 'hotspot84020', 'hotspot84029', 'hotspot84046', 'hotspot84080', 'hotspot8411', 'hotspot84133', 'hotspot84135', 'hotspot8429', 'hotspot84332', 'hotspot84424', 'hotspot84430', 'hotspot845', 'hotspot84588', 'hotspot8462', 'hotspot84639', 'hotspot84657', 'hotspot84662', 'hotspot84781', 'hotspot84805', 'hotspot84822', 'hotspot84921', 'hotspot84965', 'hotspot85016', 'hotspot85150', 'hotspot85201', 'hotspot85249', 'hotspot85293', 'hotspot85387', 'hotspot85474', 'hotspot85537', 'hotspot8562', 'hotspot85677', 'hotspot85850', 'hotspot85855', 'hotspot85910', 'hotspot85973', 'hotspot85981', 'hotspot86035', 'hotspot86057', 'hotspot86067', 'hotspot86133', 'hotspot86184', 'hotspot8635', 'hotspot86360', 'hotspot86466', 'hotspot86522', 'hotspot86531', 'hotspot86623', 'hotspot86658', 'hotspot8666', 'hotspot86757', 'hotspot86779', 'hotspot86801', 'hotspot86935', 'hotspot86996', 'hotspot86997', 'hotspot87022', 'hotspot87133', 'hotspot87169', 'hotspot87257', 'hotspot87336', 'hotspot87443', 'hotspot8759', 'hotspot87618', 'hotspot87695', 'hotspot87725', 'hotspot87951', 'hotspot88006', 'hotspot88040', 'hotspot88267', 'hotspot88428', 'hotspot88446', 'hotspot88448', 'hotspot88479', 'hotspot88538', 'hotspot8864', 'hotspot88679', 'hotspot88683', 'hotspot88709', 'hotspot88777', 'hotspot88906', 'hotspot88930', 'hotspot89001', 'hotspot8908', 'hotspot89118', 'hotspot89269', 'hotspot89287', 'hotspot89291', 'hotspot89493', 'hotspot89538', 'hotspot89671', 'hotspot89681', 'hotspot89698', 'hotspot89782', 'hotspot89886', 'hotspot89904', 'hotspot89956', 'hotspot90053', 'hotspot90136', 'hotspot90288', 'hotspot90325', 'hotspot90377', 'hotspot90384', 'hotspot90448', 'hotspot90570', 'hotspot90663', 'hotspot90839', 'hotspot90929', 'hotspot90968', 'hotspot90980', 'hotspot91012', 'hotspot91135', 'hotspot91222', 'hotspot91297', 'hotspot91392', 'hotspot91443', 'hotspot91467', 'hotspot9150', 'hotspot91663', 'hotspot91711', 'hotspot91770', 'hotspot91878', 'hotspot91891', 'hotspot9197', 'hotspot92091', 'hotspot92133', 'hotspot92186', 'hotspot92283', 'hotspot9246', 'hotspot92472', 'hotspot92529', 'hotspot92531', 'hotspot92567', 'hotspot92576', 'hotspot92589', 'hotspot92610', 'hotspot92719', 'hotspot928', 'hotspot92923', 'hotspot92934', 'hotspot93056', 'hotspot93087', 'hotspot93111', 'hotspot93184', 'hotspot93200', 'hotspot93234', 'hotspot93257', 'hotspot93265', 'hotspot93280', 'hotspot93346', 'hotspot93388', 'hotspot9339', 'hotspot93415', 'hotspot93468', 'hotspot93483', 'hotspot93494', 'hotspot93534', 'hotspot93552', 'hotspot93714', 'hotspot93807', 'hotspot93933', 'hotspot9409', 'hotspot94209', 'hotspot94297', 'hotspot94482', 'hotspot94494', 'hotspot94500', 'hotspot94553', 'hotspot94554', 'hotspot94564', 'hotspot94576', 'hotspot94591', 'hotspot94650', 'hotspot94665', 'hotspot94703', 'hotspot94728', 'hotspot94765', 'hotspot94865', 'hotspot94900', 'hotspot9521', 'hotspot95276', 'hotspot95283', 'hotspot95470', 'hotspot95474', 'hotspot95551', 'hotspot95609', 'hotspot95611', 'hotspot95647', 'hotspot95651', 'hotspot95801', 'hotspot95842', 'hotspot95850', 'hotspot9586', 'hotspot95880', 'hotspot96093', 'hotspot96133', 'hotspot96238', 'hotspot96252', 'hotspot96421', 'hotspot9650', 'hotspot96555', 'hotspot96589', 'hotspot96610', 'hotspot96716', 'hotspot9708', 'hotspot97129', 'hotspot97133', 'hotspot97254', 'hotspot97296', 'hotspot97446', 'hotspot97528', 'hotspot9756', 'hotspot97682', 'hotspot97784', 'hotspot97854', 'hotspot97913', 'hotspot9802', 'hotspot9803', 'hotspot98071', 'hotspot98124', 'hotspot98260', 'hotspot98280', 'hotspot98306', 'hotspot98327', 'hotspot98341', 'hotspot98369', 'hotspot98377', 'hotspot98401', 'hotspot98432', 'hotspot98438', 'hotspot98447', 'hotspot98601', 'hotspot98673', 'hotspot98676', 'hotspot98689', 'hotspot98760', 'hotspot98912', 'hotspot98995', 'hotspot9905', 'hotspot99215', 'hotspot99372', 'hotspot99501', 'hotspot99536', 'hotspot99592', 'hotspot9969', 'hotspot99708', 'hotspot99779', 'hotspot99809', 'hotspot99841', 'hotspot99943', 'hotspot99949'
]]
# suspect_gene_patterns = [ 'MCM[0-9]', 'SMC[0-9]', 'IFI.*', 'Hist1*.', 'HSP90.+' , 'COX.+']
suspect_gene_patterns = ['MCM[0-9]', 'SMC[0-9]', 'IFI.*', 'Hist1*.', 'HSP*' , 'COX.+']
suspect_genes_mask = mc.tl.find_named_genes(clean, names=suspect_gene_names,
                                            patterns=suspect_gene_patterns)
suspect_gene_names = sorted(clean.var_names[suspect_genes_mask])

In [ ]:
%%time
mc.pl.relate_genes(clean, random_seed=123456, 
                   genes_similarity_method="pearson")#, min_genes_of_modules=)
# method in ("pearson", "repeated_pearson", "logistics", "logistics_pearson")

In [ ]:
%%time
module_of_genes = clean.var['related_genes_module']

In [ ]:
%%time
module_of_genes = clean.var['related_genes_module']
suspect_gene_modules = np.unique(module_of_genes[suspect_genes_mask])
suspect_gene_modules = suspect_gene_modules[suspect_gene_modules >= 0]
print(suspect_gene_modules)

In [ ]:
%%time
similarity_of_genes = mc.ut.get_vv_frame(clean, 'related_genes_similarity')
for gene_module in range(max(module_of_genes)+1):
# for gene_module in range(55,56):
    module_genes_mask = module_of_genes == gene_module
    similarity_of_module = similarity_of_genes.loc[module_genes_mask, module_genes_mask]
    similarity_of_module.index = \
    similarity_of_module.columns = [
        '(*) ' + name if name in suspect_gene_names else name
        for name in similarity_of_module.index
    ]
    ax = plt.axes()
    sb.heatmap(similarity_of_module, vmin=0, vmax=1, ax=ax, cmap="YlGnBu", xticklabels=True, yticklabels=True)
#     gm_perc = np.percentile(mc.ut.slice(clean, vars=module_genes_mask).X.sum(axis=1)/clean.X.sum(axis=1), 99.9)
    ax.set_title(f'Gene Module {gene_module}')
    if gene_module in suspect_gene_modules:
        plt.savefig('embs_s_{}.png'.format(gene_module), bbox_inches='tight')
    else:
        plt.savefig('embs_{}.png'.format(gene_module), bbox_inches='tight')
    plt.show()

In [ ]:
%%time
forbidden_genes_mask = suspect_genes_mask

# for gene_module in [
#     #1,2,3,4,5,6,10,11,12,13,14,15,16,17,18,22,23,25,27,28,29,30,31,32,36,37,39,41,42,43,45,49,50,56,60,61,62,64,65,66,70,73,75,76,79,82,83,84,88,94,103,106,131,140,146,147,156,164,
#     177, # m_genes_177
#     181, # S_genes_181
#     186,193,212,
# ]:
for gene_module in [0,1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,29,30,32,33,34,37,40,41,42,43,45,48,
                    50,52,54,55,56,58,62,65,68,70,71,73,76,87,90,121,125,135,136,141,150,152,160,161,171,174,178,179,188,
                    195,202,]:
# for gene_module in [
#     1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,22,23,25,26,27,28,29,30,31,32,33,36,37,38,39,41,42,43,44,45,49,
#     50,  # cycling
#     51,55,56,60,61,62,64,65,66,70,71,72,73,75,76,79,82,83,84,88,94,103,106,131,140,146,
#     147,  # cycling
# #     156,  # ? -
#     164,169,
# #     171,  # ? -
# #     177,  # cycling
#     181,  # cycling
#     186,  # ? +
#     193, 
# #     212,  # ? -
#     ]:
# [0, 1, 2, 3, 6, 7, 8, 9, 11, 13, 16, 17, 18, 24, 31, 33, 36, 40, 41, 45, 47, 53, 56, 61, 65, 77, 82, 83, 85, 87, 96, 108, 109, 114, 120, 123, 127, 131, 136, 138, 139, 141, 147, 152]:
# [22,57,71,72,73,75,79]:
# [43,61,66,75,81,84,86,89,94,]:
# [1,5,36,37,48,59,64,65,74,75,79,84]:
# [0, 1, 2, 6, 8, 10, 26, 29, 31, 43, 53, 56, 60, 66, 76, 79, 89, 91, 92, 105, 106, 112, 114, 120, 121, 122, 125, 127, 130, 139, 140, 142, 145, 146]:
# [17, 19, 27, 30, 32, 38, 50, 52, 55, 56, 57, 59, 62, 63, 65, 68, 69, 74, 76, 78, 79, 82, 83, 84, 89, 90, 91, 92, 98, 103, 110, 113]:
# [9, 21, 33, 34, 38, 47, 49, 66, 70, 76, 78, 79, 81]: # embs85_wl_nohs
# [7, 25, 43, 44, 58, 63, 70, 74, 77, 85, 91, 92, 106, 123, 126, 127]:  # embs85_wl
# [16, 33, 40, 42, 44, 45, 46, 63, 71, 75, 77, 81, 83, 89, 90, 99, 104, 106]: #embs_batch85
# [1, 5, 6,    12,     17,         30, 32,     45,     47, 51, 54,         68,         72, 75,             80, ]: #embs85_ss yoav 
#     [1, 6, 8, 12, 13, 17, 19, 28, 30, 32, 36, 46, 47, 51, 54, 68, 70, 72, 75, 78, 80]: #embs85_ss
#     [5, 7, 8, 12, 13, 16, 17, 20, 21, 24, 33, 35, 36, 42, 45, 51, 55, 56, 57, 59, 60, 85, 89, 100, 103, 
#      108, 114, 120, 124, 126, 127, 132, 133, 141, 145, 149]: #embs85
    module_genes_mask = module_of_genes == gene_module
    forbidden_genes_mask |= module_genes_mask
# forbidden_genes_mask['HES1'] = False
forbidden_gene_names = sorted(clean.var_names[forbidden_genes_mask])
print(len(forbidden_gene_names))
print(' '.join(forbidden_gene_names))

In [ ]:
max_parallel_piles = mc.pl.guess_max_parallel_piles(clean)
print(max_parallel_piles)
mc.pl.set_max_parallel_piles(max_parallel_piles)

In [ ]:
with mc.ut.progress_bar():
    mc.pl.divide_and_conquer_pipeline(clean,
                                      forbidden_gene_names=forbidden_gene_names,
                                      target_metacell_size=320000,
                                      random_seed=123456)

In [ ]:
%%time
metacells = mc.pl.collect_metacells(clean, name='metacells')

In [ ]:
metacells.obs_names

In [ ]:
min(mc.ut.slice(clean, obs=clean.obs.metacell >= 0).obs.groupby('metacell').aggregate('count')['properly_sampled_cell'])

In [ ]:
max(mc.ut.slice(clean, obs=clean.obs.metacell >= 0).obs.groupby('metacell').aggregate('count')['properly_sampled_cell'])

In [ ]:
%%time
mc.pl.compute_umap_by_features(metacells, max_top_feature_genes=1000,
                               min_dist=2.0, random_seed=123456)

In [ ]:
umap_x = mc.ut.get_o_numpy(metacells, 'umap_x')
umap_y = mc.ut.get_o_numpy(metacells, 'umap_y')
plot = sb.scatterplot(x=umap_x, y=umap_y)

In [ ]:
%%time
umap_edges = sp.coo_matrix(mc.ut.get_oo_proper(metacells, 'obs_outgoing_weights'))
min_long_edge_size = 4
sb.set()
plot = sb.scatterplot(x=umap_x, y=umap_y)
for (source_index, target_index, weight) \
        in zip(umap_edges.row, umap_edges.col, umap_edges.data):
    source_x = umap_x[source_index]
    target_x = umap_x[target_index]
    source_y = umap_y[source_index]
    target_y = umap_y[target_index]
    if hypot(target_x - source_x, target_y - source_y) >= min_long_edge_size:
        plt.plot([source_x, target_x], [source_y, target_y],
                 linewidth=weight * 2, color='indigo')
plt.show()

In [ ]:
# np.median(np.sum(metacells.X, axis=1))

In [ ]:
# clean.X.shape

In [ ]:
# len(clean.obs_names), len(set(clean.obs_names))

In [ ]:
clean.write('cells11_clean_2203.h5ad')
metacells.write('metacells11_2203.h5ad')
# del metacells.uns['__name__']
# metacells.write('for_seurat_ss5.h5ad')

In [ ]:
all_hs = set([g for g in clean.var[clean.var['related_genes_module'] > -1].index if 'hotspot' in g])
# interesting_hs = all_hs - set(forbidden_gene_names) 
interesting_hs = {'hotspot655',}

In [ ]:
blast6 = pd.read_table('blasted_all_refseq6', header=None)
interesting_blast6 = blast6[blast6[0].isin(interesting_hs)]

In [ ]:
interesting_blast6['max11'] = interesting_blast6.groupby(0)[11].transform(max)
top_blast6 = interesting_blast6[interesting_blast6[11]==interesting_blast6['max11']]

In [ ]:
iii = [n for n in interesting_blast6[1].values]

In [ ]:
import mygene
mg = mygene.MyGeneInfo()
ret = mg.querymany(iii, scopes='refseq')

In [ ]:
set([i['symbol'] for i in ret if 'symbol' in i])

In [ ]:
metacells_fhs = ad.read_h5ad('metacells11_fhs.h5ad')
clean = ad.read_h5ad('cells11_clean_2203.h5ad')

In [ ]:
cdata = mc.ut.slice(clean, vars=metacells_fhs.var_names)

In [ ]:
outliers = mc.pl.compute_for_mcview(adata=cdata, gdata=metacells_fhs, compute_umap_by_features_2=None, )

In [ ]:
outliers.write('outliers11_2203_for_mcview.h5ad')

In [ ]:
metacells_fhs.write('metacells11_2203_for_mcview.h5ad')

In [ ]:
metacells_fhs =  ad.read_h5ad('metacells11_2203_for_mcview_ron.h5ad')

In [ ]:
proj = ad.read_h5ad('metacells11_2203_projected_on_mouse.h5ad')

In [ ]:
proj.layers

In [ ]:
adata = ad.read_h5ad('/home/ofirr/amos_ofirr/embflow3/metacells_embexe2_computed_for_mcview11.h5ad')

In [ ]:
adata.var.index = adata.var.index.str.upper()

In [ ]:
mc.ut.set_name(adata, "atlas.metacells")
mc.ut.set_name(metacells_fhs, "query.metacells")

In [ ]:
len(set(adata.var.index) & set(metacells_fhs.var.index))

In [ ]:
weights, qdata = mc.pl.projection_pipeline(adata=adata, qdata=metacells_fhs, reproducible=True)

In [ ]:
qdata.write_h5ad('metacells11_2203_projected_on_mouse.h5ad')

In [ ]:
with open("rabemb2203_11.on.mouse.query.atlas.weights.csv", "w") as file:
    file.write("query,atlas,weight\n")
    for query_index, atlas_index, in zip(*weights.nonzero()):
        weight = weights[query_index, atlas_index]
        file.write(f"{query_index},{atlas_index},{weight}\n")

In [ ]:
adata_rab = ad.read_h5ad('metacells10_2203_projected_on_rabemb2112_ron_ron_annotated.h5ad')
metacells_fhs = ad.read_h5ad('metacells11_2203_for_mcview_ron.h5ad')

In [ ]:
# types = pd.Series(
#     pd.read_csv('metacell_types-2022-03-27_withexe.csv').cell_type, 
#     dtype="category"
# )
# types.index = adata_rab.obs.index
# adata_rab.obs['type'] = types

In [ ]:
# adata_rab.write_h5ad('metacells10_2203_projected_on_rabemb2112_ron_ron_annotated.h5ad')

In [ ]:
mc.ut.set_name(adata_rab, "atlas.metacells")
mc.ut.set_name(metacells_fhs, "query.metacells")

In [ ]:
len(set(adata_rab.var.index) & set(metacells_fhs.var.index))

In [ ]:
weights, qdata = mc.pl.projection_pipeline(adata=adata_rab, qdata=metacells_fhs, reproducible=True)

In [ ]:
qdata.write_h5ad('metacells_2203_11_projected_on_rabemb_2203_10.h5ad')

In [ ]:
with open("rabemb2203_11.on.rabemb2203_10.query.atlas.weights.csv", "w") as file:
    file.write("query,atlas,weight\n")
    for query_index, atlas_index, in zip(*weights.nonzero()):
        weight = weights[query_index, atlas_index]
        file.write(f"{query_index},{atlas_index},{weight}\n")

In [ ]:
import anndata as ad
ADATA_FILE_PATH = "/home/ofirr/amos_ofirr/rabemb_2203/metacells_2203_11_projected_on_rabemb_2203_10.h5ad"
input_data = ad.read_h5ad(ADATA_FILE_PATH)

In [ ]:
input_data

In [ ]:
exe_mcs = [635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 2164, 2165, 2166, 2167, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175, 2176, 2177, 2178, 2179, 2180, 2181, 2182, 2183, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 3336, 3338, 3341, 3386, 3389, 3400, 3405, 3407, 3415, 3416, 3424, 3425, 3430, 3438, 3440, 3446, 3447, 3462, 3470, 3471, 3473, 3477, 3482, 3514, 3520, 3527, 3535, 3536, 3550, 3551, 3556, 3558, 3567, 3577, 3591, 3596, 3607, 3608, 3613, 3614, 3627, 3645, 3647, 3649, 3653, 3663, 3664, 3667, 3669, 3670, 3671, 3674, 3675, 3676, 3681, 3695, 3697, 3698, 3700, 3702, 3708]
exe_mcs = [str(i) for i in exe_mcs]

In [ ]:
exec_mcs_11 = [479, 482, 486, 489, 490, 496, 499, 503, 510, 511, 515, 527, 535, 539, 547, 549, 554, 559, 562, 567, 571, 613, 615, 617, 639, 643, 647, 648, 655, 666, 683, 691, 695, 696, 698, 708, 712, 713, 715, 717, 718, 719, 3282, 3395, 3437, 3533, 477, 478, 485, 497, 502, 505, 506, 507, 509, 512, 518, 519, 521, 523, 525, 526, 532, 537, 538, 540, 543, 545, 546, 551, 553, 557, 558, 564, 565, 566, 568, 573, 574, 575, 577, 578, 580, 582, 584, 592, 594, 597, 598, 599, 600, 601, 602, 604, 605, 606, 608, 609, 611, 616, 618, 620, 621, 623, 629, 630, 631, 634, 636, 637, 638, 640, 641, 642, 645, 650, 651, 652, 653, 654, 660, 663, 664, 668, 671, 672, 673, 674, 677, 703, 723, 724, 1365, 1381, 1384, 1469, 1491, 1545, 3316, 3381, 3386, 3416, 3426, 3474, 3482, 3515, 3522, 3531, 481, 487, 491, 492, 493, 494, 495, 498, 500, 501, 516, 517, 522, 524, 528, 529, 530, 533, 536, 542, 548, 550, 552, 555, 556, 560, 561, 563, 576, 579, 581, 583, 585, 587, 589, 591, 595, 607, 619, 622, 624, 627, 628, 632, 633, 635, 644, 656, 657, 659, 662, 667, 669, 670, 676, 678, 681, 682, 684, 685, 687, 690, 694, 701, 704, 706, 710, 711, 714, 716, 720, 721, 722, 3292, 3331, 3357, 3402, 3446, 3552, 3568, 480, 483, 484, 504, 508, 513, 514, 520, 531, 534, 541, 544, 569, 570, 572, 588, 590, 596, 603, 610, 612, 614, 625, 626, 646, 649, 658, 661, 675, 1251, 1252, 1255, 1256, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1282, 1283, 1285, 1286, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1300, 1301, 1302, 1303, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1314, 1315, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1325, 1326, 1328, 1329, 1330, 1332, 1333, 1335, 1336, 1338, 1339, 1340, 1341, 1342, 1343, 1345, 1346, 1347, 1348, 1349, 1350, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1360, 1361, 1362, 1363, 1364, 1366, 1367, 1368, 1369, 1370, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1382, 1383, 1385, 1386, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1427, 1428, 1429, 1430, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1461, 1462, 1463, 1464, 1465, 1467, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1492, 1493, 1494, 1495, 1496, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1513, 1531, 1533, 1536, 1539, 1540, 1541, 1547, 1549, 1554, 1557, 1558, 3259, 3266, 3268, 3284, 3290, 3304, 3306, 3327, 3360, 3364, 3388, 3418, 3429, 3438, 3441, 3444, 3445, 3449, 3462, 3469, 3490, 3496, 3500, 3519, 3529, 1254, 1281, 1287, 1299, 1313, 1331, 1334, 1337, 1344, 1351, 1359, 1372, 1397, 1405, 1415, 1416, 1431, 1439, 1466, 1468, 1481, 1497, 1505, 1509, 1510, 1514, 1515, 1517, 1518, 1520, 1526, 1527, 1528, 1530, 1532, 1534, 1537, 1538, 1555, 1556, 3335, 3439, 3535, 3557, 3583, 1506, 1507, 1512, 1516, 1519, 1522, 1523, 1535, 1542, 1543, 1544, 1546, 1548, 1550, 1551, 1552, 1553, 3572, 3573, 488, 586, 665, 679, 680, 686, 688, 689, 692, 693, 697, 699, 700, 702, 705, 707, 709, 1253, 1257, 1268, 1284, 1304, 1316, 1324, 1327, 1387, 1426, 1460, 1508, 1511, 1521, 1524, 1525, 1529, 3283, 3326, 3344, ]

In [ ]:
len(exec_mcs_11)

In [ ]:
import metacells as mc
execs = mc.ut.slice(input_data, obs=(input_data.obs.metacell.isin(exec_mcs_11)))

In [ ]:
input_data2 = mc.ut.slice(input_data, obs=(input_data.obs.index.isin(cells_with_meta.obs.index)))

In [ ]:
input_data2.obs.embryo = cells_with_meta.obs.embryo[input_data2.obs.index]

In [ ]:
import pandas as pd
annots = pd.read_csv('metacell_types-2022-03-27_withexe.csv')

In [ ]:
annots

In [ ]:
input_data2.obs = input_data2.obs.join(annots, on='metacell', lsuffix='a')

In [ ]:
emb_cols = Counter(zip(input_data2.obs.embryo, input_data2.obs.color))

In [ ]:
df = pd.DataFrame([{'embryo':k[0], 'color':k[1], 'count':v} for k, v in emb_cols.items()])

In [ ]:
emb_sums = df.groupby('embryo').sum().to_dict()['count']

In [ ]:
df.to_csv('embs_cols.csv', )

In [ ]:
df['ncount'] = [c/emb_sums[e] for c, e in zip(df['count'], df.embryo)]

In [ ]:
df.groupby(['embryo','color', 'ncount']).sum().unstack().plot(kind='bar', stacked=True)

In [ ]:
# require('anndata')
# require('chameleon')
# require('pheatmap')
# require('pracma')
# require('stats')
# options(jupyter.plot_mimetypes = 'image/svg+xml')

In [ ]:
# mc.ut.get_o_names(metacells)

In [ ]:
# mc.ut.get_

In [ ]:
# mc.ut.get_m_data(metacells)

In [ ]:
# mc.ut.get_v_names(metacells)

In [ ]:
# #markus test
# clean = ad.read_h5ad('embs_hs_clean.h5ad')
# metacells = ad.read_h5ad('metacells.h5ad')

In [ ]:
fet = mc.ut.slice(metacells, vars=metacells.var.feature_gene > 0)

In [ ]:
fet_hotspots = mc.ut.slice(fet, vars=fet.var_names.isin({i for i in fet.var_names if 'hotspot' in i}))

In [ ]:
fet_hotspots.X.shape

In [ ]:
fet.X.shape

In [ ]:
410/1659

In [ ]:
np.max(fet_hotspots.X, axis=0)

In [ ]:
# notcox = np.log(1e-5+mc.ut.slice(metacells, vars=metacells.var_names != 'COX2').X)

In [ ]:
# cox = np.log2(1e-5+mc.ut.slice(metacells, vars=metacells.var_names == 'COX2').X)

In [ ]:
# cox.T.shape, notcox.T.shape

In [ ]:
# cors = np.corrcoef(notcox.T, cox.T )

In [ ]:
# eomes = np.log2(1e-5+pd.DataFrame.sparse.from_spmatrix(mc.ut.slice(clean, vars=clean.var_names == 'EOMES').X).values)

In [ ]:
# import seaborn as sns

# plt.hist(eomes, bins=20)
# plt.show()

In [ ]:
# mc.ut.slice(clean, vars=clean.var_names == 'EOMES',
#             obs=np.array(
#     (pd.DataFrame.sparse.from_spmatrix(mc.ut.slice(clean, vars=clean.var_names == 'EOMES').X)[0] > 20).values
# )).obs

In [ ]:
# mc.ut.slice(clean, vars=clean.var_names == 'EOMES').X

In [ ]:
# [(i,j) for i,j in pd.DataFrame.sparse.from_spmatrix(mc.ut.slice(clean, vars=clean.var_names == 'EOMES').X)[0].items() if j > 20]

In [ ]:
# sus_mc_clus = [573, 574, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 599, 600, 601, 602, 603, 604, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 636, 637, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 691, 695, 696, 697, 699, 700, 703, 704, 705, 706, 1280, 1303, 1308, 1359, 1362, 1368, 1369]

In [ ]:
# clean.obs_keys()

In [ ]:
# hs = mc.ut.slice(metacells, vars=metacells.var_names.str.contains('hotspot'))
# nhs = mc.ut.slice(metacells, vars=~metacells.var_names.str.contains('hotspot'))


In [ ]:
# ths = np.squeeze(np.asarray(np.sum(hs.X, axis = 1)))

In [ ]:
# tnhs = np.squeeze(np.asarray(np.sum(nhs.X, axis = 1)))

In [ ]:
# plt.hist(tnhs/(ths+tnhs), bins=100)
# plt.show()

In [ ]:
# import seaborn as sns

# sns.kdeplot(tnhs/(ths+tnhs))
# plt.show()